In [1]:
#instalo/actualizo
#! pip3 install --upgrade google-cloud-aiplatform


In [5]:
import re
import pandas as pd
from google.cloud import bigquery
#import vertexai 
#from vertexai.generative_models import GenerationConfig, GenerativeModel, Image, Part, Content
#import vertexai.preview.generative_models as generative_models 
import sys
import os
from fuzzywuzzy import process, fuzz

In [6]:
import os 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:\\Users\\flori\\OneDrive\\Desktop\\LDP\\ldp-data-genai-producto-b871ebffc681.json"

In [7]:
# Crear un cliente de BigQuery con el proyecto especificado
client = bigquery.Client(project='ldp-data-wyscout')


PARA EMPEZAR ME TRAIGO LAS TABLAS DE TRANSFERENCIAS DE TFMKT Y WYSCOUT

In [8]:


# Consulta SQL
query2 = """SELECT * FROM ldp-data-wyscout.transfer_market.players_transfers_curated"""

try:
    # Ejecutar la consulta y obtener los resultados en un DataFrame
    df1 = client.query(query2).to_dataframe()
    print(df1.head())
except Exception as e:
    print(f"An error occurred: {e}")


  row_ind      player_name season transfer_date        team_from  \
0       1  Alexander Nübel  25/26    2026-06-30    VfB Stuttgart   
1      14    Rubén Alcaraz  04/05    2004-07-01      Barça Youth   
2      21    Andy Lonergan  02/03    2002-12-27       Darlington   
3       8          Willian  05/06    2006-01-01  Corinthians U20   
4       2        Guilherme  02/03    2003-01-01             PSTC   

          team_to market_value transfer_value contract_expiry  \
0   Bayern Munich      1.2e+07             NA      2026-06-30   
1  Espanyol Youth           NA              0              NA   
2         Preston           NA             NA              NA   
3     Corinthians           NA             NA              NA   
4     Atlético-PR           NA             NA              NA   

                                          player_url  query_date  \
0  https://www.transfermarkt.com/alexander-nubel/...  2024-09-10   
1  https://www.transfermarkt.com/ruben-alcaraz/pr...  2024-09-07

In [6]:
#df1=pd.read_csv('tfmkt.csv')

In [6]:
df1.head(20)

,row_ind,player_name,season,transfer_date,team_from,team_to,market_value,transfer_value,contract_expiry,player_url,query_date,timestamp,blob,player_id,teams_url,team_id,player_transfer_id
0,9,Joel Monteiro,05/06,2005-07-01,Salgueiros Y15,Salgueiros Y17,NA,NA,NA,https://www.transfermarkt.com/joel-monteiro/pr...,2024-09-09,2024-09-07 14:31:16+00:00,transfermarkt/player_transfers/ players_transf...,248983,https://www.transfermarkt.com/amora-fc/startse...,1414,200507011414248983
1,18,Andy Lonergan,05/06,2005-10-06,Preston,Wycombe,4e+05,NA,NA,https://www.transfermarkt.com/andy-lonergan/pr...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,14044,https://www.transfermarkt.com/fc-everton/start...,29,200510062914044
2,7,Peter Pekarik,04/05,2004-07-08,MSK Zilina,Dubnica,NA,NA,NA,https://www.transfermarkt.com/peter-pekarik/pr...,2024-09-10,2024-09-07 14:32:10+00:00,transfermarkt/player_transfers/ players_transf...,51100,https://www.transfermarkt.com/hertha-bsc/start...,44,200407084451100
3,6,Peter Pekarik,04/05,2005-06-30,Dubnica,MSK Zilina,NA,NA,NA,https://www.transfermarkt.com/peter-pekarik/pr...,2024-09-10,2024-09-07 14:32:10+00:00,transfermarkt/player_transfers/ players_transf...,51100,https://www.transfermarkt.com/hertha-bsc/start...,44,200506304451100
4,9,Roberto,05/06,2005-12-01,Paysandu,Santos B,NA,NA,NA,https://www.transfermarkt.com/roberto/profil/s...,2024-09-11,2024-09-07 14:32:10+00:00,transfermarkt/player_transfers/ players_transf...,202006,https://www.transfermarkt.com/kitchee/startsei...,15979,2005120115979202006
5,23,Francesco Mautone,04/05,2004-07-01,FC Bolzano 1996,Ferentino,NA,NA,NA,https://www.transfermarkt.com/francesco-mauton...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,407680,https://www.transfermarkt.com/palmese/startsei...,62357,2004070162357407680
6,19,Haris Seferovic,05/06,2005-07-01,Luzern U16,Luzern U17,NA,NA,NA,https://www.transfermarkt.com/haris-seferovic/...,2024-09-09,2024-09-07 14:31:16+00:00,transfermarkt/player_transfers/ players_transf...,109256,https://www.transfermarkt.com/celta-vigo/start...,940,20050701940109256
7,18,Felipe Ramos,05/06,2005-07-01,Real Madrid U17,Real Madrid U18,NA,NA,NA,https://www.transfermarkt.com/felipe-ramos/pro...,2024-09-11,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,59568,https://www.transfermarkt.com/fc-cordoba/start...,993,2005070199359568
8,8,Bernhard Marx,05/06,2006-04-03,Hallein 04 Jgd.,USK Anif Youth,NA,NA,NA,https://www.transfermarkt.com/bernhard-marx/pr...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,334361,https://www.transfermarkt.com/sk-adnet/startse...,26456,2006040326456334361
9,21,Adam Le Fondre,04/05,2004-07-01,Stockport U18,Stockport,NA,NA,NA,https://www.transfermarkt.com/adam-le-fondre/p...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,46106,https://www.transfermarkt.com/sydney-fc/starts...,8054,20040701805446106


In [7]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1101741 entries, 0 to 1101740
Data columns (total 17 columns):
 #   Column              Non-Null Count    Dtype              
---  ------              --------------    -----              
 0   row_ind             1101741 non-null  object             
 1   player_name         1101741 non-null  object             
 2   season              1101741 non-null  object             
 3   transfer_date       1101741 non-null  object             
 4   team_from           1101741 non-null  object             
 5   team_to             1101741 non-null  object             
 6   market_value        1101741 non-null  object             
 7   transfer_value      1101741 non-null  object             
 8   contract_expiry     1101741 non-null  object             
 9   player_url          1101741 non-null  object             
 10  query_date          1101741 non-null  dbdate             
 11  timestamp           1101741 non-null  datetime64[us, UTC]
 12  

In [9]:
#dftext = df1.to_csv('tfmkt.csv', index=False)

In [8]:
# Contar valores nulos en df1
null_counts_df1 = df1.isnull().sum()
print("Valores nulos en df1:")
print(null_counts_df1)



Valores nulos en df1:
row_ind                   0
player_name               0
season                    0
transfer_date             0
team_from                 0
team_to                   0
market_value              0
transfer_value            0
contract_expiry           0
player_url                0
query_date                0
timestamp                 0
blob                  96968
player_id                 1
teams_url                 1
team_id                   1
player_transfer_id        1
dtype: int64


In [9]:
###FILTRO LA DE DICT TEAMS POR team_type="parent" and category="default" and gender = 'male' Y ADEMAS POR LOS PAISES QUE ESTAN EN LA DE CAPOLOGY PARA ACHICAR REGISTROS

query = """select * from ldp-data-wyscout.wyscout_curated.transfer_con_nombre;
 """

try:
    # Ejecutar la consulta y obtener los resultados en un DataFrame                     ###domestic_competition_name
    df2 = client.query(query).to_dataframe()
    print(df2.head())
except Exception as e:
    print(f"An error occurred: {e}")

                                  nombre  transferId  fromTeamId  \
0                    Hans Kristian Heier     2252031        7133   
1                         Sanel Ibisevic     2251975        7352   
2                      Dominik Petrovčin     2401863       24150   
3                            Leon Graham     2351286       10860   
4  Matias Inojosa Galindo Nunes Teixeira     2352901       31422   

              fromTeamName  toTeamId                      toTeamName active  \
0        Drøbak / Frogn IL      7191                Lillestrøm SK II      0   
1              Follo FK II     34765            TSV Havelse Under 19      0   
2     HNK Šibenik Under 19     34096  FC Internazionale Milano Under      0   
3          Cliftonville FC      <NA>                            None      0   
4  Gil Vicente FC Under 19      <NA>                            None      0   

    startDate     endDate        type value currency announceDate  playerId  
0  2022-01-04  2023-08-21    Transfer 

In [10]:
# Contar valores nulos en df2
null_counts_df2 = df2.isnull().sum()
print("Valores nulos en df2:")
print(null_counts_df2)

Valores nulos en df2:
nombre              77
transferId           0
fromTeamId      593112
fromTeamName    198586
toTeamId        293553
toTeamName      229523
active               0
startDate         1026
endDate         696105
type                55
value                0
currency          1395
announceDate         0
playerId             0
dtype: int64


In [13]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1933826 entries, 0 to 1933825
Data columns (total 14 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   nombre        object
 1   transferId    Int64 
 2   fromTeamId    Int64 
 3   fromTeamName  object
 4   toTeamId      Int64 
 5   toTeamName    object
 6   active        object
 7   startDate     object
 8   endDate       object
 9   type          object
 10  value         object
 11  currency      object
 12  announceDate  object
 13  playerId      Int64 
dtypes: Int64(4), object(10)
memory usage: 213.9+ MB


In [11]:
# Filtrar las filas donde fromTeamId es nulo
null_fromTeamId_rows = df2[df2['fromTeamId'].isnull()]

# Mostrar las filas con valores nulos en fromTeamId
null_fromTeamId_rows.head(30)

,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,endDate,type,value,currency,announceDate,playerId
6,Yassen Khaled Mousa Mohammad,2251563,<NA>,,16584,Al Jalil,0,2024-01-01,None,Free Transfer,0,EUR,,1061798
7,Herbton Ferreira da Silva,2251693,<NA>,,5721,CS Sergipe,0,2024-03-01,None,Free Transfer,0,EUR,,1061861
8,Torpal Ali Abdukorimovitsj Salamov,2251889,<NA>,,7141,Kvik Halden,0,2024-03-21,None,Free Transfer,0,EUR,,1061891
9,Andreas Ramstad,2251898,<NA>,,7141,Kvik Halden,0,2024-03-12,None,Free Transfer,0,EUR,,1061892
10,Anders Molund,2251904,<NA>,,7141,Kvik Halden,0,2024-02-05,None,Free Transfer,0,EUR,,1061894
11,Davis Kvants,2252411,<NA>,,75651,Ogre United,0,2024-01-01,None,Free Transfer,0,EUR,,1062000
12,Giorgi Macharashvili,2252418,<NA>,,75651,Ogre United,0,2024-01-01,None,Free Transfer,0,EUR,,1062006
13,Ricards Dimants,2252409,<NA>,,75651,Ogre United,0,2024-01-01,None,Free Transfer,0,EUR,,1061998
14,Vladimirs Paskovics,2252412,<NA>,,75651,Ogre United,0,2024-01-01,None,Free Transfer,0,EUR,,1062001
15,Sharpy Njobvu,2252143,<NA>,,17645,Green Eagles,0,2024-01-01,None,Free Transfer,0,EUR,,1061945


In [14]:
# Convert columns with type object to string
df2 = df2.astype({col: 'string' for col in df2.select_dtypes(include='object').columns})

In [16]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1933826 entries, 0 to 1933825
Data columns (total 14 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   nombre        string
 1   transferId    Int64 
 2   fromTeamId    Int64 
 3   fromTeamName  string
 4   toTeamId      Int64 
 5   toTeamName    string
 6   active        string
 7   startDate     string
 8   endDate       string
 9   type          string
 10  value         string
 11  currency      string
 12  announceDate  string
 13  playerId      Int64 
dtypes: Int64(4), string(10)
memory usage: 213.9 MB


In [12]:

# Eliminar filas con valores nulos en df2
df2_clean = df2.dropna()

print("\nDataFrame df2_clean:")
print(df2_clean.head())


DataFrame df2_clean:
                 nombre  transferId  fromTeamId                 fromTeamName  \
3         Lucas Poulsen     2251604       34555                    Sundby BK   
6   Herman Oen Paulsrud     2252078        7194            Stabæk Fotball II   
11         Osama Housni     2252129        7354                Nordstrand IF   
32      Ivan Zheleznyak     1382703       31313  FC Olimpik Donetsk Under 19   
33    Bektur Abdyvaliev     1185550       18646            FK Dordoi Bishkek   

    toTeamId                    toTeamName active   startDate     endDate  \
3       7463                  BK Frem 1886      0  2024-02-01  2024-06-30   
6      24654          Grorud Idrettslag II      0  2022-06-29  2024-03-22   
11      7345                     Grorud IL      0  2024-03-05  2024-08-16   
32     66970  FC Inhulets Petrove Under 19      0  2021-07-30  2022-07-01   
33     62586                     FC Ilbirs      0  2021-03-11  2022-12-31   

             type value currency a

In [13]:
df2_clean.head(20)

,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,endDate,type,value,currency,announceDate,playerId
3,Lucas Poulsen,2251604,34555,Sundby BK,7463,BK Frem 1886,0,2024-02-01,2024-06-30,Transfer,0,EUR,,1061847
6,Herman Oen Paulsrud,2252078,7194,Stabæk Fotball II,24654,Grorud Idrettslag II,0,2022-06-29,2024-03-22,Transfer,0,EUR,,1061921
11,Osama Housni,2252129,7354,Nordstrand IF,7345,Grorud IL,0,2024-03-05,2024-08-16,Free Transfer,0,EUR,,1061932
32,Ivan Zheleznyak,1382703,31313,FC Olimpik Donetsk Under 19,66970,FC Inhulets Petrove Under 19,0,2021-07-30,2022-07-01,Transfer,0,EUR,,685491
33,Bektur Abdyvaliev,1185550,18646,FK Dordoi Bishkek,62586,FC Ilbirs,0,2021-03-11,2022-12-31,Loan,0,EUR,,685546
35,Frederik Kyed Thomsen,1199479,7622,Kolding IF,7499,SønderjyskE,0,2021-01-01,2024-01-01,Transfer,0,EUR,,685608
37,Jie Fu,1930012,33192,Heilongjiang Ice City FC,75640,Shaanxi Union FC,0,2023-04-20,2024-02-22,Transfer,0,EUR,,685650
38,Matteo Spada,2103305,34497,Alcione Milano,36279,USD Castellanzese 1921,0,2023-07-01,2024-07-01,Transfer,0,EUR,,685515
40,Judith Roosjen,1427746,369,SC Heerenveen,598,VV Alkmaar,0,2021-07-01,2023-07-01,Transfer,0,EUR,,685448
41,Emir Yıldız,1469878,67471,Adiyaman 1954 Spor,40073,Adıyaman 1954 Spor Kulübü,0,2021-08-03,2022-07-01,Transfer,0,EUR,,685579


In [14]:
df2_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 854637 entries, 3 to 1933825
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   nombre        854637 non-null  object
 1   transferId    854637 non-null  Int64 
 2   fromTeamId    854637 non-null  Int64 
 3   fromTeamName  854637 non-null  object
 4   toTeamId      854637 non-null  Int64 
 5   toTeamName    854637 non-null  object
 6   active        854637 non-null  object
 7   startDate     854637 non-null  object
 8   endDate       854637 non-null  object
 9   type          854637 non-null  object
 10  value         854637 non-null  object
 11  currency      854637 non-null  object
 12  announceDate  854637 non-null  object
 13  playerId      854637 non-null  Int64 
dtypes: Int64(4), object(10)
memory usage: 101.1+ MB


In [17]:
# Ordenar df1 alfabéticamente por la columna 'player_name'
df1_sorted = df1.sort_values(by='player_name')
print("df1 ordenado alfabéticamente por 'player_name':")




df1 ordenado alfabéticamente por 'player_name':


In [18]:
df1_sorted.head(50)

,row_ind,player_name,season,transfer_date,team_from,team_to,market_value,transfer_value,contract_expiry,player_url,query_date,timestamp,blob,player_id,teams_url,team_id,player_transfer_id
1014713,1,- Stewart,23/24,2023-07-01,Gala Fairydean,Without Club,NA,NA,NA,https://www.transfermarkt.com/-stewart/profil/...,2024-09-11,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,1092926,https://www.transfermarkt.com/gala-fairydean-r...,11379,20230701113791092926
618081,1,. Bana,20/21,2020-07-01,Own Youth,Desp. Huíla,NA,NA,NA,https://www.transfermarkt.com/-bana/profil/spi...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,887419,https://www.transfermarkt.com/desportivo-da-hu...,39419,2020070139419887419
947482,1,A Mohamed Liyaakath,23/24,2024-02-13,Without Club,G. Kerala B,25000,NA,NA,https://www.transfermarkt.com/a-mohamed-liyaak...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,842536,https://www.transfermarkt.com/chennaiyin-fc-ii...,67620,2024021367620842536
1051843,2,A Mohamed Liyaakath,23/24,2023-06-01,Chennaiyin FC,Without Club,25000,NA,NA,https://www.transfermarkt.com/a-mohamed-liyaak...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,842536,https://www.transfermarkt.com/chennaiyin-fc-ii...,67620,2023060167620842536
615348,4,A Mohamed Liyaakath,20/21,2020-11-01,Chennaiyin U16,Indian Arrows,NA,0,NA,https://www.transfermarkt.com/a-mohamed-liyaak...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,842536,https://www.transfermarkt.com/chennaiyin-fc-ii...,67620,2020110167620842536
712225,3,A Mohamed Liyaakath,21/22,2021-11-01,Indian Arrows,Chennaiyin FC,25000,0,NA,https://www.transfermarkt.com/a-mohamed-liyaak...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,842536,https://www.transfermarkt.com/chennaiyin-fc-ii...,67620,2021110167620842536
892525,1,A Ranjethkumar,23/24,2023-06-01,Chennaiyin II,Without Club,NA,NA,NA,https://www.transfermarkt.com/a-ranjethkumar/p...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,1118356,https://www.transfermarkt.com/chennaiyin-fc-ii...,67620,20230601676201118356
942165,1,A San,23/24,2023-09-01,Binh Duong FC,Hai Phong FC,50000,0,NA,https://www.transfermarkt.com/a-san/profil/spi...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,560526,https://www.transfermarkt.com/becamex-binh-duo...,10756,2023090110756560526
508570,5,A San,19/20,2019-12-31,HCMC FC,HAGL FC,NA,NA,NA,https://www.transfermarkt.com/a-san/profil/spi...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,560526,https://www.transfermarkt.com/becamex-binh-duo...,10756,2019123110756560526
468589,6,A San,18/19,2019-01-01,HAGL FC,HCMC FC,NA,NA,NA,https://www.transfermarkt.com/a-san/profil/spi...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,560526,https://www.transfermarkt.com/becamex-binh-duo...,10756,2019010110756560526


In [19]:
# Ordenar df2 alfabéticamente por la columna 'nombre'
df2_sorted = df2.sort_values(by='nombre')
print("\ndf2 ordenado alfabéticamente por 'nombre':")



df2 ordenado alfabéticamente por 'nombre':


In [20]:
df2_sorted.head(50)

,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,endDate,type,value,currency,announceDate,playerId
1634707,,1442529,<NA>,<NA>,12461,Keshla FK,0,2015-01-01,2016-09-30,Free Transfer,0,EUR,,667752
1689115,,1453486,12461,Keshla FK,<NA>,<NA>,0,2016-09-30,<NA>,Free Agent,0,EUR,,667753
1564808,,1442550,<NA>,<NA>,12461,Keshla FK,0,2015-01-01,2016-09-30,Free Transfer,0,EUR,,667753
1695485,,1453484,12461,Keshla FK,<NA>,<NA>,0,2016-09-30,<NA>,Free Agent,0,EUR,,667752
521990,Bohdan Yatselenko,1453100,<NA>,,68937,VIK-Volun U15,0,2021-07-01,<NA>,Free Transfer,0,EUR,,776941
440468,AJ DePinto,1888873,<NA>,,72205,Seattle Pacific Falcon Univers,0,2023-01-01,2023-07-01,Free Transfer,0,EUR,,906828
484858,AJ DePinto,1996935,72205,Seattle Pacific Falcon Univers,<NA>,<NA>,0,2023-07-01,<NA>,Free Agent,0,EUR,,906828
1198737,AJ Valdivia,2431908,<NA>,<NA>,71041,California State San Bernardin,0,2024-08-01,<NA>,Free Transfer,0,EUR,,1117560
1689642,ARIANA KING,1399149,<NA>,,61788,Houston Baptist Huskies Univer,0,2021-07-01,2022-06-30,Free Transfer,0,EUR,,765187
1689676,ARIANA KING,1692598,61788,Houston Baptist Huskies Univer,<NA>,<NA>,0,2022-06-30,<NA>,Free Agent,0,EUR,,765187


In [22]:
#df2=pd.read_csv('wyscout.csv')

In [23]:
# Reemplazar NaN con un valor por defecto, por ejemplo, -1 o 0
#df2['fromTeamId'] = df2['fromTeamId'].fillna(0).astype(int)
#df2['toTeamId'] = df2['toTeamId'].fillna(0).astype(int)


In [21]:
df2.head()

,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,endDate,type,value,currency,announceDate,playerId
0,Hans Kristian Heier,2252031,7133,Drøbak / Frogn IL,7191,Lillestrøm SK II,0,2022-01-04,2023-08-21,Transfer,0,EUR,,1061911
1,Sanel Ibisevic,2251975,7352,Follo FK II,34765,TSV Havelse Under 19,0,2017-08-24,2018-09-04,Transfer,0,EUR,,1061902
2,Dominik Petrovčin,2401863,24150,HNK Šibenik Under 19,34096,FC Internazionale Milano Under,0,2024-08-28,<NA>,Transfer,0,EUR,,1061786
3,Leon Graham,2351286,10860,Cliftonville FC,<NA>,<NA>,0,2024-06-30,<NA>,Free Agent,0,EUR,,1061851
4,Matias Inojosa Galindo Nunes Teixeira,2352901,31422,Gil Vicente FC Under 19,<NA>,<NA>,0,2024-06-30,<NA>,Free Agent,0,EUR,,1061797


In [25]:
#dftext2 = df2.to_csv('wyscout.csv',index=False)

In [22]:
# Tomar las primeras 1000 filas de df1_sorted y df2_sorted
df1_sorted_top1000 = df1_sorted.head(1000)
df2_sorted_top1000 = df2_sorted.head(1000)


In [23]:
# Tomar las primeras 5000 filas de df1_sorted y df2_sorted
df1_sorted_top5000 = df1_sorted.head(5000)
df2_sorted_top5000 = df2_sorted.head(5000)

# Mostrar las primeras filas para verificar
print(df1_sorted_top5000.head())
print(df2_sorted_top5000.head())

        row_ind          player_name season transfer_date       team_from  \
1014713       1            - Stewart  23/24    2023-07-01  Gala Fairydean   
618081        1               . Bana  20/21    2020-07-01       Own Youth   
947482        1  A Mohamed Liyaakath  23/24    2024-02-13    Without Club   
1051843       2  A Mohamed Liyaakath  23/24    2023-06-01   Chennaiyin FC   
615348        4  A Mohamed Liyaakath  20/21    2020-11-01  Chennaiyin U16   

               team_to market_value transfer_value contract_expiry  \
1014713   Without Club           NA             NA              NA   
618081     Desp. Huíla           NA             NA              NA   
947482     G. Kerala B        25000             NA              NA   
1051843   Without Club        25000             NA              NA   
615348   Indian Arrows           NA              0              NA   

                                                player_url  query_date  \
1014713  https://www.transfermarkt.com/-st

In [24]:
df1_sorted_top1000.head(20)

,row_ind,player_name,season,transfer_date,team_from,team_to,market_value,transfer_value,contract_expiry,player_url,query_date,timestamp,blob,player_id,teams_url,team_id,player_transfer_id
1014713,1,- Stewart,23/24,2023-07-01,Gala Fairydean,Without Club,NA,NA,NA,https://www.transfermarkt.com/-stewart/profil/...,2024-09-11,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,1092926,https://www.transfermarkt.com/gala-fairydean-r...,11379,20230701113791092926
618081,1,. Bana,20/21,2020-07-01,Own Youth,Desp. Huíla,NA,NA,NA,https://www.transfermarkt.com/-bana/profil/spi...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,887419,https://www.transfermarkt.com/desportivo-da-hu...,39419,2020070139419887419
947482,1,A Mohamed Liyaakath,23/24,2024-02-13,Without Club,G. Kerala B,25000,NA,NA,https://www.transfermarkt.com/a-mohamed-liyaak...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,842536,https://www.transfermarkt.com/chennaiyin-fc-ii...,67620,2024021367620842536
1051843,2,A Mohamed Liyaakath,23/24,2023-06-01,Chennaiyin FC,Without Club,25000,NA,NA,https://www.transfermarkt.com/a-mohamed-liyaak...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,842536,https://www.transfermarkt.com/chennaiyin-fc-ii...,67620,2023060167620842536
615348,4,A Mohamed Liyaakath,20/21,2020-11-01,Chennaiyin U16,Indian Arrows,NA,0,NA,https://www.transfermarkt.com/a-mohamed-liyaak...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,842536,https://www.transfermarkt.com/chennaiyin-fc-ii...,67620,2020110167620842536
712225,3,A Mohamed Liyaakath,21/22,2021-11-01,Indian Arrows,Chennaiyin FC,25000,0,NA,https://www.transfermarkt.com/a-mohamed-liyaak...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,842536,https://www.transfermarkt.com/chennaiyin-fc-ii...,67620,2021110167620842536
892525,1,A Ranjethkumar,23/24,2023-06-01,Chennaiyin II,Without Club,NA,NA,NA,https://www.transfermarkt.com/a-ranjethkumar/p...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,1118356,https://www.transfermarkt.com/chennaiyin-fc-ii...,67620,20230601676201118356
942165,1,A San,23/24,2023-09-01,Binh Duong FC,Hai Phong FC,50000,0,NA,https://www.transfermarkt.com/a-san/profil/spi...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,560526,https://www.transfermarkt.com/becamex-binh-duo...,10756,2023090110756560526
508570,5,A San,19/20,2019-12-31,HCMC FC,HAGL FC,NA,NA,NA,https://www.transfermarkt.com/a-san/profil/spi...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,560526,https://www.transfermarkt.com/becamex-binh-duo...,10756,2019123110756560526
468589,6,A San,18/19,2019-01-01,HAGL FC,HCMC FC,NA,NA,NA,https://www.transfermarkt.com/a-san/profil/spi...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,560526,https://www.transfermarkt.com/becamex-binh-duo...,10756,2019010110756560526


In [25]:
df2_sorted_top1000.head(20)

,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,endDate,type,value,currency,announceDate,playerId
1634707,,1442529,<NA>,<NA>,12461,Keshla FK,0,2015-01-01,2016-09-30,Free Transfer,0,EUR,,667752
1689115,,1453486,12461,Keshla FK,<NA>,<NA>,0,2016-09-30,<NA>,Free Agent,0,EUR,,667753
1564808,,1442550,<NA>,<NA>,12461,Keshla FK,0,2015-01-01,2016-09-30,Free Transfer,0,EUR,,667753
1695485,,1453484,12461,Keshla FK,<NA>,<NA>,0,2016-09-30,<NA>,Free Agent,0,EUR,,667752
521990,Bohdan Yatselenko,1453100,<NA>,,68937,VIK-Volun U15,0,2021-07-01,<NA>,Free Transfer,0,EUR,,776941
440468,AJ DePinto,1888873,<NA>,,72205,Seattle Pacific Falcon Univers,0,2023-01-01,2023-07-01,Free Transfer,0,EUR,,906828
484858,AJ DePinto,1996935,72205,Seattle Pacific Falcon Univers,<NA>,<NA>,0,2023-07-01,<NA>,Free Agent,0,EUR,,906828
1198737,AJ Valdivia,2431908,<NA>,<NA>,71041,California State San Bernardin,0,2024-08-01,<NA>,Free Transfer,0,EUR,,1117560
1689642,ARIANA KING,1399149,<NA>,,61788,Houston Baptist Huskies Univer,0,2021-07-01,2022-06-30,Free Transfer,0,EUR,,765187
1689676,ARIANA KING,1692598,61788,Houston Baptist Huskies Univer,<NA>,<NA>,0,2022-06-30,<NA>,Free Agent,0,EUR,,765187


In [28]:
import pandas as pd
import unicodedata
from fuzzywuzzy import fuzz

In [29]:

def normalize_text(text):
    if isinstance(text, str):
        # Convertir a minúsculas, normalizar acentos y eliminar exceso de espacios
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
        text = ' '.join(text.lower().strip().split())
        return text
    return text

def simplify_name(name):
    parts = name.split()
    # Retorna sólo el primer y último nombre
    return ' '.join([parts[0], parts[-1]]) if len(parts) > 1 else name

def simplify_club_name(club_name):
    # Convierte a minúsculas y elimina palabras comunes (ajusta según tus necesidades)
    ignore_words = {'fc', 'club', 'de', 'the'}
    parts = club_name.lower().split()
    return ' '.join([word for word in parts if word not in ignore_words])

# Cargar los archivos CSV en dataframes
#df1_sorted_top1000 = pd.read_csv('archivo1.csv')
#df2_sorted_top5000 = pd.read_csv('archivo2.csv')

# Normalizar y simplificar columnas relevantes
df1_sorted_top1000['player_name'] = df1_sorted_top1000['player_name'].apply(normalize_text).apply(simplify_name)
df1_sorted_top1000['team_from'] = df1_sorted_top1000['team_from'].apply(normalize_text).apply(simplify_club_name)
df1_sorted_top1000['team_to'] = df1_sorted_top1000['team_to'].apply(normalize_text).apply(simplify_club_name)
df1_sorted_top1000['transfer_date'] = pd.to_datetime(df1_sorted_top1000['transfer_date'], errors='coerce')

df2_sorted_top1000['nombre'] = df2_sorted_top1000['nombre'].apply(normalize_text).apply(simplify_name)
df2_sorted_top1000['fromTeamName'] = df2_sorted_top1000['fromTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted_top1000['toTeamName'] = df2_sorted_top1000['toTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted_top1000['startDate'] = pd.to_datetime(df2_sorted_top1000['startDate'], errors='coerce')

# Crear un ID temporal basado en mes y año
df1_sorted_top1000['ID'] = df1_sorted_top1000.apply(lambda row: f"{row['team_from']}_{row['team_to']}_{row['transfer_date'].year}_{row['transfer_date'].month}", axis=1)
df2_sorted_top1000['ID'] = df2_sorted_top1000.apply(lambda row: f"{row['fromTeamName']}_{row['toTeamName']}_{row['startDate'].year}_{row['startDate'].month}", axis=1)

# Unión basada en IDs aproximados y similitud de nombres
merged_rows = []

for idx1, row1 in df1_sorted_top1000.iterrows():
    for idx2, row2 in df2_sorted_top1000.iterrows():
        if row1['ID'] == row2['ID']:
            # Usar similitud fuzzywuzzy para nombre del jugador
            name_similarity = fuzz.partial_ratio(row1['player_name'], row2['nombre'])
            # Verifica que la similitud supere un umbral (ajustable según necesidad)
            if name_similarity > 80:
                merged_rows.append(row1.to_dict() | row2.to_dict())

# Crear un dataframe con los resultados de la unión
merged_df22 = pd.DataFrame(merged_rows)

# Ver los resultados
print(merged_df22.head())

# Guardar a un archivo CSV
merged_df22.to_csv('archivo_unidomil.csv', index=False)


C:\Users\flori\AppData\Local\Temp\ipykernel_14580\1259585350.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_sorted_top1000['player_name'] = df1_sorted_top1000['player_name'].apply(normalize_text).apply(simplify_name)
C:\Users\flori\AppData\Local\Temp\ipykernel_14580\1259585350.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_sorted_top1000['team_from'] = df1_sorted_top1000['team_from'].apply(normalize_text).apply(simplify_club_name)
C:\Users\flori\AppData\Local\Temp\ipykernel_14580\1259585

  row_ind           player_name season transfer_date       team_from  \
0       3            aj cochran  20/21    2021-02-17  phoenix rising   
1       2         aaren d'silva  20/21    2020-10-28          goa ii   
2       6  aaron amadi-holloway  20/21    2020-10-17   brisbane roar   
3       2  aaron amadi-holloway  22/23    2022-08-05   burton albion   
4       5  aaron amadi-holloway  21/22    2021-07-23  sc east bengal   

          team_to market_value transfer_value contract_expiry  \
0     indy eleven       250000              0              NA   
1             goa        75000             NA              NA   
2  sc east bengal        2e+05              0              NA   
3        hereford        2e+05              0              NA   
4   burton albion        2e+05              0              NA   

                                          player_url  ... toTeamId  \
0  https://www.transfermarkt.com/aj-cochran/profi...  ...    27170   
1  https://www.transfermarkt.com/aar

In [34]:
merged_df22.head(20)

,row_ind,player_name,season,transfer_date,team_from,team_to,market_value,transfer_value,contract_expiry,player_url,query_date,timestamp,blob,player_id,teams_url,team_id,player_transfer_id,ID,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,endDate,type,value,currency,announceDate,playerId
0,3,aj cochran,20/21,2021-02-17,phoenix rising,indy eleven,250000,0,NA,https://www.transfermarkt.com/aj-cochran/profi...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,306391,https://www.transfermarkt.com/charleston-batte...,4115,202102174115306391,phoenix rising_indy eleven_2021_2,aj cochran,1156285,29992,phoenix rising,27170,indy eleven,0,2021-02-17,2023-02-02,Transfer,0,EUR,,300553
1,2,aaren d'silva,20/21,2020-10-28,goa ii,goa,75000,NA,NA,https://www.transfermarkt.com/aaren-dsilva/pro...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,630382,https://www.transfermarkt.com/hyderabad-fc/sta...,76364,2020102876364630382,goa ii_goa_2020_10,aaren d'silva,1059236,61377,goa ii,31884,goa,0,2020-10-28,2021-06-18,Transfer,0,EUR,,703579
2,6,aaron amadi-holloway,20/21,2020-10-17,brisbane roar,sc east bengal,2e+05,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2020101746842186574,brisbane roar_sc east bengal_2020_10,aaron amadi-holloway,1054694,8314,brisbane roar,15036,sc east bengal,0,2020-10-17,2021-07-23,Free Transfer,0,EUR,,10612
3,2,aaron amadi-holloway,22/23,2022-08-05,burton albion,hereford,2e+05,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2022080546842186574,burton albion_hereford_2022_8,aaron amadi-holloway,1674586,1710,burton albion,36605,hereford,0,2022-08-05,2023-07-12,Transfer,0,EUR,,10612
4,5,aaron amadi-holloway,21/22,2021-07-23,sc east bengal,burton albion,2e+05,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2021072346842186574,sc east bengal_burton albion_2021_7,aaron amadi-holloway,1350346,15036,sc east bengal,1710,burton albion,0,2021-07-23,2022-01-28,Transfer,0,EUR,,10612
5,15,aaron amadi-holloway,11/12,2012-01-01,bath city,bristol city,50000,NA,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2012010146842186574,bath city_bristol city_2012_1,aaron amadi-holloway,1572374,1728,bath city,1655,bristol city,0,2012-01-01,2014-02-12,Back from Loan,0,EUR,,10612
6,16,aaron amadi-holloway,11/12,2011-11-24,bristol city,bath city,50000,NA,2012-06-30,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2011112446842186574,bristol city_bath city_2011_11,aaron amadi-holloway,80644,1655,bristol city,1728,bath city,0,2011-11-24,2012-01-01,Loan,0,,2011-11-26,10612
7,7,aaron arnott,22/23,2022-09-07,raith rovers,brechin city,125000,NA,2024-05-31,https://www.transfermarkt.com/aaron-arnott/pro...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,822785,https://www.transfermarkt.com/brechin-city-fc/...,3194,202209073194822785,raith rovers_brechin city_2022_9,aaron arnott,1771043,8527,raith rovers,8529,brechin city,0,2022-09-07,2023-01-13,Transfer,0,EUR,,699833
8,6,aaron arnott,22/23,2023-01-13,brechin city,raith rovers,125000,NA,2023-05-31,https://www.transfermarkt.com/aaron-arnott/pro...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_tra

In [37]:
merged_df22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   row_ind             27 non-null     object             
 1   player_name         27 non-null     object             
 2   season              27 non-null     object             
 3   transfer_date       27 non-null     datetime64[ns]     
 4   team_from           27 non-null     object             
 5   team_to             27 non-null     object             
 6   market_value        27 non-null     object             
 7   transfer_value      27 non-null     object             
 8   contract_expiry     27 non-null     object             
 9   player_url          27 non-null     object             
 10  query_date          27 non-null     object             
 11  timestamp           27 non-null     datetime64[ns, UTC]
 12  blob                27 non-null     ob

In [52]:

# Funciones auxiliares para la normalización
def normalize_text(text):
    if isinstance(text, str):
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
        return ' '.join(text.lower().strip().split())
    return text

def simplify_name(name):
    parts = name.split()
    return ' '.join([parts[0], parts[-1]]) if len(parts) > 1 else name

def simplify_club_name(club_name):
    ignore_words = {'fc', 'club', 'de', 'the'}
    return ' '.join([word for word in club_name.lower().split() if word not in ignore_words])

# Normalizar y simplificar columnas relevantes
df1_sorted_top1000['player_name'] = df1_sorted_top1000['player_name'].apply(normalize_text).apply(simplify_name)
df1_sorted_top1000['team_from'] = df1_sorted_top1000['team_from'].apply(normalize_text).apply(simplify_club_name)
df1_sorted_top1000['team_to'] = df1_sorted_top1000['team_to'].apply(normalize_text).apply(simplify_club_name)
df1_sorted_top1000['transfer_date'] = pd.to_datetime(df1_sorted_top1000['transfer_date'], errors='coerce')

df2_sorted_top1000['nombre'] = df2_sorted_top1000['nombre'].apply(normalize_text).apply(simplify_name)
df2_sorted_top1000['fromTeamName'] = df2_sorted_top1000['fromTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted_top1000['toTeamName'] = df2_sorted_top1000['toTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted_top1000['startDate'] = pd.to_datetime(df2_sorted_top1000['startDate'], errors='coerce')

# Crear un ID temporal basado en mes y año
df1_sorted_top1000['ID'] = df1_sorted_top1000.apply(lambda row: f"{row['team_from']}_{row['team_to']}_{row['transfer_date'].year}_{row['transfer_date'].month}", axis=1)
df2_sorted_top1000['ID'] = df2_sorted_top1000.apply(lambda row: f"{row['fromTeamName']}_{row['toTeamName']}_{row['startDate'].year}_{row['startDate'].month}", axis=1)

# Primaria unión por ID
initial_merge = pd.merge(df1_sorted_top1000, df2_sorted_top1000, on='ID', suffixes=('_df1', '_df2'))

# Unión final basada en similitud
merged_rows = []

for _, merged_row in initial_merge.iterrows():
    name_similarity = fuzz.partial_ratio(merged_row['player_name'], merged_row['nombre'])
    if name_similarity > 80:
        merged_rows.append(merged_row)

# Crear un dataframe final con los resultados
merged_dfmil = pd.DataFrame(merged_rows)

# Ver los resultados
print(merged_dfmil.head())

# Guardar el resultado en un archivo CSV
merged_dfmil.to_csv('archivo_unido.csv', index=False)

  row_ind           player_name season transfer_date       team_from  \
0       3            aj cochran  20/21    2021-02-17  phoenix rising   
1       2         aaren d'silva  20/21    2020-10-28          goa ii   
2       6  aaron amadi-holloway  20/21    2020-10-17   brisbane roar   
3       2  aaron amadi-holloway  22/23    2022-08-05   burton albion   
4       5  aaron amadi-holloway  21/22    2021-07-23  sc east bengal   

          team_to market_value transfer_value contract_expiry  \
0     indy eleven       250000              0              NA   
1             goa        75000             NA              NA   
2  sc east bengal        2e+05              0              NA   
3        hereford        2e+05              0              NA   
4   burton albion        2e+05              0              NA   

                                          player_url  query_date  \
0  https://www.transfermarkt.com/aj-cochran/profi...  2024-09-10   
1  https://www.transfermarkt.com/aaren-d

C:\Users\flori\AppData\Local\Temp\ipykernel_14580\2920607803.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_sorted_top1000['player_name'] = df1_sorted_top1000['player_name'].apply(normalize_text).apply(simplify_name)
C:\Users\flori\AppData\Local\Temp\ipykernel_14580\2920607803.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_sorted_top1000['team_from'] = df1_sorted_top1000['team_from'].apply(normalize_text).apply(simplify_club_name)
C:\Users\flori\AppData\Local\Temp\ipykernel_14580\2920607

In [53]:
merged_dfmil.head(20)

,row_ind,player_name,season,transfer_date,team_from,team_to,market_value,transfer_value,contract_expiry,player_url,query_date,timestamp,blob,player_id,teams_url,team_id,player_transfer_id,ID,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,endDate,type,value,currency,announceDate,playerId
0,3,aj cochran,20/21,2021-02-17,phoenix rising,indy eleven,250000,0,NA,https://www.transfermarkt.com/aj-cochran/profi...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,306391,https://www.transfermarkt.com/charleston-batte...,4115,202102174115306391,phoenix rising_indy eleven_2021_2,aj cochran,1156285,29992,phoenix rising,27170,indy eleven,0,2021-02-17,2023-02-02,Transfer,0,EUR,,300553
1,2,aaren d'silva,20/21,2020-10-28,goa ii,goa,75000,NA,NA,https://www.transfermarkt.com/aaren-dsilva/pro...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,630382,https://www.transfermarkt.com/hyderabad-fc/sta...,76364,2020102876364630382,goa ii_goa_2020_10,aaren d'silva,1059236,61377,goa ii,31884,goa,0,2020-10-28,2021-06-18,Transfer,0,EUR,,703579
2,6,aaron amadi-holloway,20/21,2020-10-17,brisbane roar,sc east bengal,2e+05,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2020101746842186574,brisbane roar_sc east bengal_2020_10,aaron amadi-holloway,1054694,8314,brisbane roar,15036,sc east bengal,0,2020-10-17,2021-07-23,Free Transfer,0,EUR,,10612
3,2,aaron amadi-holloway,22/23,2022-08-05,burton albion,hereford,2e+05,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2022080546842186574,burton albion_hereford_2022_8,aaron amadi-holloway,1674586,1710,burton albion,36605,hereford,0,2022-08-05,2023-07-12,Transfer,0,EUR,,10612
4,5,aaron amadi-holloway,21/22,2021-07-23,sc east bengal,burton albion,2e+05,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2021072346842186574,sc east bengal_burton albion_2021_7,aaron amadi-holloway,1350346,15036,sc east bengal,1710,burton albion,0,2021-07-23,2022-01-28,Transfer,0,EUR,,10612
5,15,aaron amadi-holloway,11/12,2012-01-01,bath city,bristol city,50000,NA,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2012010146842186574,bath city_bristol city_2012_1,aaron amadi-holloway,1572374,1728,bath city,1655,bristol city,0,2012-01-01,2014-02-12,Back from Loan,0,EUR,,10612
6,16,aaron amadi-holloway,11/12,2011-11-24,bristol city,bath city,50000,NA,2012-06-30,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2011112446842186574,bristol city_bath city_2011_11,aaron amadi-holloway,80644,1655,bristol city,1728,bath city,0,2011-11-24,2012-01-01,Loan,0,,2011-11-26,10612
7,7,aaron arnott,22/23,2022-09-07,raith rovers,brechin city,125000,NA,2024-05-31,https://www.transfermarkt.com/aaron-arnott/pro...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,822785,https://www.transfermarkt.com/brechin-city-fc/...,3194,202209073194822785,raith rovers_brechin city_2022_9,aaron arnott,1771043,8527,raith rovers,8529,brechin city,0,2022-09-07,2023-01-13,Transfer,0,EUR,,699833
8,6,aaron arnott,22/23,2023-01-13,brechin city,raith rovers,125000,NA,2023-05-31,https://www.transfermarkt.com/aaron-arnott/pro...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_tra

In [54]:
# Realizar una unión completa (full outer join) basada en el ID
merged_full_df = pd.merge(df1_sorted_top1000, df2_sorted_top1000, on='ID', how='outer', suffixes=('_df1', '_df2'))

# Ver los resultados
merged_full_df.head(10)



,row_ind,player_name,season,transfer_date,team_from,team_to,market_value,transfer_value,contract_expiry,player_url,query_date,timestamp,blob,player_id,teams_url,team_id,player_transfer_id,ID,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,endDate,type,value,currency,announceDate,playerId
0,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,1. kaiserslautern ii_tus rot-weiss koblenz_2022_8,jin eine,1709509,2596,1. kaiserslautern ii,32421,tus rot-weiss koblenz,0,2022-08-01,2023-07-01,Transfer,0,EUR,,845054
1,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,1. kaiserslautern under 17_1. kaiserslautern u...,aaron basenach,999034,32898,1. kaiserslautern under 17,3093,1. kaiserslautern under 19,0,2020-07-01,2022-11-22,Transfer,0,EUR,,686995
2,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,1. kaiserslautern under 19_1. kaiserslautern_2...,aaron basenach,1807189,3093,1. kaiserslautern under 19,2456,1. kaiserslautern,0,2022-11-22,2024-07-25,Transfer,0,EUR,,686995
3,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,1. koln ii_sg dynamo dresden_2024_6,jonas saliger,1961365,2576,1. koln ii,2478,sg dynamo dresden,0,2024-06-30,2024-07-01,Back from Loan,0,EUR,,687378
4,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,1. magdeburg under 19_tsv havelse under 19_2022_6,irichad behrens,1725156,25856,1. magdeburg under 19,34765,tsv havelse under 19,0,2022-06-30,2023-07-01,Transfer,0,EUR,,687466
5,2,a. darar,22/23,2022-07-01,1.fc duren jgd.,f. cologne u17,NA,0,NA,https://www.transfermarkt.com/a-darar/profil/s...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,1041468,https://www.transfermarkt.com/sc-fortuna-koln-...,34114,20220701341141041468,1.fc duren jgd._f. cologne u17_2022_7,NaN,<NA>,<NA>,NaN,<NA>,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,<NA>
6,1,a. kalayci,24/25,2024-07-01,1.fc duren u19,1.fc duren ii,NA,NA,NA,https://www.transfermarkt.com/a-kalayci/profil...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,1056949,https://www.transfermarkt.com/bonner-sc-u19/st...,7781,2024070177811056949,1.fc duren u19_1.fc duren ii_2024_7,NaN,<NA>,<NA>,NaN,<NA>,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,<NA>
7,1,aaron frey,22/23,2022-07-01,1.fc erlensee,sg barockstadt,NA,0,NA,https://www.transfermarkt.com/aaron-frey/profi...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,369705,https://www.transfermarkt.com/sg-barockstadt-f...,66588,2022070166588369705,1.fc erlensee_sg barockstadt_2022_7,NaN,<NA>,<NA>,NaN,<NA>,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,<NA>
8,1,aaron basenach,24/25,2024-07-25,1.fc k'lautern,fk pirmasens,1e+05,0,NA,https://www.transfermarkt.com/aaron-basenach/p...,2024-09-07,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,798092,https://www.transfermarkt.com/1-fc-kaiserslaut...,2,202407252798092,1.fc k'lautern_fk pirmasens_2024_7,NaN,<NA>,<NA>,NaN,<NA>,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,<NA>
9,2,aaron bayakala,22/23,2022-07-01,1.fc koln u17,1.fc koln u19,NA,NA,2024-06-30,https://www.transfermarkt.com/aaron-bayakala/p...,2024-09-11,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,917931,https://www.transfermarkt.com/1-fc-koln-u19/st...,1461,202207011461917931,1.fc koln u17_1.fc koln u19_2022_7,NaN,<NA>,<NA>,NaN,<NA>,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,<NA>


Pruebo unir 5 mil a partir de generar un id con nomralizacion de texto y simplificacion de nmombre y de club

In [55]:

#pruebo la uinion con el id con 5 mil filas
# Funciones auxiliares para la normalización
def normalize_text(text):
    if isinstance(text, str):
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
        return ' '.join(text.lower().strip().split())
    return text

def simplify_name(name):
    parts = name.split()
    return ' '.join([parts[0], parts[-1]]) if len(parts) > 1 else name

def simplify_club_name(club_name):
    ignore_words = {'fc', 'club', 'de', 'the'}
    return ' '.join([word for word in club_name.lower().split() if word not in ignore_words])

# Normalizar y simplificar columnas relevantes
df1_sorted_top5000['player_name'] = df1_sorted_top5000['player_name'].apply(normalize_text).apply(simplify_name)
df1_sorted_top5000['team_from'] = df1_sorted_top5000['team_from'].apply(normalize_text).apply(simplify_club_name)
df1_sorted_top5000['team_to'] = df1_sorted_top5000['team_to'].apply(normalize_text).apply(simplify_club_name)
df1_sorted_top5000['transfer_date'] = pd.to_datetime(df1_sorted_top5000['transfer_date'], errors='coerce')

df2_sorted_top5000['nombre'] = df2_sorted_top5000['nombre'].apply(normalize_text).apply(simplify_name)
df2_sorted_top5000['fromTeamName'] = df2_sorted_top5000['fromTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted_top5000['toTeamName'] = df2_sorted_top5000['toTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted_top5000['startDate'] = pd.to_datetime(df2_sorted_top5000['startDate'], errors='coerce')

# Crear un ID temporal basado en mes y año
df1_sorted_top5000['ID'] = df1_sorted_top5000.apply(lambda row: f"{row['team_from']}_{row['team_to']}_{row['transfer_date'].year}_{row['transfer_date'].month}", axis=1)
df2_sorted_top5000['ID'] = df2_sorted_top5000.apply(lambda row: f"{row['fromTeamName']}_{row['toTeamName']}_{row['startDate'].year}_{row['startDate'].month}", axis=1)

# Primaria unión por ID
initial_merge = pd.merge(df1_sorted_top5000, df2_sorted_top5000, on='ID', suffixes=('_df1', '_df2'))

# Unión final basada en similitud
merged_rows = []

for _, merged_row in initial_merge.iterrows():
    name_similarity = fuzz.partial_ratio(merged_row['player_name'], merged_row['nombre'])
    if name_similarity > 80:
        merged_rows.append(merged_row)

# Crear un dataframe final con los resultados
merged_df = pd.DataFrame(merged_rows)

# Ver los resultados
print(merged_df.head())

# Guardar el resultado en un archivo CSV
merged_df.to_csv('archivo_unido.csv', index=False)


  row_ind           player_name season transfer_date       team_from  \
0       3            aj cochran  20/21    2021-02-17  phoenix rising   
1       2         aaren d'silva  20/21    2020-10-28          goa ii   
2       6  aaron amadi-holloway  20/21    2020-10-17   brisbane roar   
3       2  aaron amadi-holloway  22/23    2022-08-05   burton albion   
4       5  aaron amadi-holloway  21/22    2021-07-23  sc east bengal   

          team_to market_value transfer_value contract_expiry  \
0     indy eleven       250000              0              NA   
1             goa        75000             NA              NA   
2  sc east bengal        2e+05              0              NA   
3        hereford        2e+05              0              NA   
4   burton albion        2e+05              0              NA   

                                          player_url  query_date  \
0  https://www.transfermarkt.com/aj-cochran/profi...  2024-09-10   
1  https://www.transfermarkt.com/aaren-d

C:\Users\flori\AppData\Local\Temp\ipykernel_14580\3078297709.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_sorted_top5000['player_name'] = df1_sorted_top5000['player_name'].apply(normalize_text).apply(simplify_name)
C:\Users\flori\AppData\Local\Temp\ipykernel_14580\3078297709.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_sorted_top5000['team_from'] = df1_sorted_top5000['team_from'].apply(normalize_text).apply(simplify_club_name)
C:\Users\flori\AppData\Local\Temp\ipykernel_14580\3078297

In [56]:

pd.set_option('display.max_columns', None)

In [57]:
merged_df.head(20)

,row_ind,player_name,season,transfer_date,team_from,team_to,market_value,transfer_value,contract_expiry,player_url,query_date,timestamp,blob,player_id,teams_url,team_id,player_transfer_id,ID,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,endDate,type,value,currency,announceDate,playerId
0,3,aj cochran,20/21,2021-02-17,phoenix rising,indy eleven,250000,0,NA,https://www.transfermarkt.com/aj-cochran/profi...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,306391,https://www.transfermarkt.com/charleston-batte...,4115,202102174115306391,phoenix rising_indy eleven_2021_2,aj cochran,1156285,29992,phoenix rising,27170,indy eleven,0,2021-02-17,2023-02-02,Transfer,0,EUR,,300553
1,2,aaren d'silva,20/21,2020-10-28,goa ii,goa,75000,NA,NA,https://www.transfermarkt.com/aaren-dsilva/pro...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,630382,https://www.transfermarkt.com/hyderabad-fc/sta...,76364,2020102876364630382,goa ii_goa_2020_10,aaren d'silva,1059236,61377,goa ii,31884,goa,0,2020-10-28,2021-06-18,Transfer,0,EUR,,703579
2,6,aaron amadi-holloway,20/21,2020-10-17,brisbane roar,sc east bengal,2e+05,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2020101746842186574,brisbane roar_sc east bengal_2020_10,aaron amadi-holloway,1054694,8314,brisbane roar,15036,sc east bengal,0,2020-10-17,2021-07-23,Free Transfer,0,EUR,,10612
3,2,aaron amadi-holloway,22/23,2022-08-05,burton albion,hereford,2e+05,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2022080546842186574,burton albion_hereford_2022_8,aaron amadi-holloway,1674586,1710,burton albion,36605,hereford,0,2022-08-05,2023-07-12,Transfer,0,EUR,,10612
4,5,aaron amadi-holloway,21/22,2021-07-23,sc east bengal,burton albion,2e+05,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2021072346842186574,sc east bengal_burton albion_2021_7,aaron amadi-holloway,1350346,15036,sc east bengal,1710,burton albion,0,2021-07-23,2022-01-28,Transfer,0,EUR,,10612
5,15,aaron amadi-holloway,11/12,2012-01-01,bath city,bristol city,50000,NA,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2012010146842186574,bath city_bristol city_2012_1,aaron amadi-holloway,1572374,1728,bath city,1655,bristol city,0,2012-01-01,2014-02-12,Back from Loan,0,EUR,,10612
6,16,aaron amadi-holloway,11/12,2011-11-24,bristol city,bath city,50000,NA,2012-06-30,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2011112446842186574,bristol city_bath city_2011_11,aaron amadi-holloway,80644,1655,bristol city,1728,bath city,0,2011-11-24,2012-01-01,Loan,0,,2011-11-26,10612
7,7,aaron arnott,22/23,2022-09-07,raith rovers,brechin city,125000,NA,2024-05-31,https://www.transfermarkt.com/aaron-arnott/pro...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,822785,https://www.transfermarkt.com/brechin-city-fc/...,3194,202209073194822785,raith rovers_brechin city_2022_9,aaron arnott,1771043,8527,raith rovers,8529,brechin city,0,2022-09-07,2023-01-13,Transfer,0,EUR,,699833
8,6,aaron arnott,22/23,2023-01-13,brechin city,raith rovers,125000,NA,2023-05-31,https://www.transfermarkt.com/aaron-arnott/pro...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_tra

In [58]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 231 entries, 0 to 250
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   row_ind             231 non-null    object             
 1   player_name         231 non-null    object             
 2   season              231 non-null    object             
 3   transfer_date       231 non-null    datetime64[ns]     
 4   team_from           231 non-null    object             
 5   team_to             231 non-null    object             
 6   market_value        231 non-null    object             
 7   transfer_value      231 non-null    object             
 8   contract_expiry     231 non-null    object             
 9   player_url          231 non-null    object             
 10  query_date          231 non-null    object             
 11  timestamp           231 non-null    datetime64[ns, UTC]
 12  blob                218 non-null    objec

In [59]:
# Funciones auxiliares para la normalización
def normalize_text(text):
    if isinstance(text, str):
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
        return ' '.join(text.lower().strip().split())
    return text

def simplify_name(name):
    parts = name.split()
    return ' '.join([parts[0], parts[-1]]) if len(parts) > 1 else name

def simplify_club_name(club_name):
    ignore_words = {'fc', 'club', 'de', 'the'}
    return ' '.join([word for word in club_name.lower().split() if word not in ignore_words])

# Normalizar y simplificar columnas relevantes
df1_sorted['player_name'] = df1_sorted['player_name'].apply(normalize_text).apply(simplify_name)
df1_sorted['team_from'] = df1_sorted['team_from'].apply(normalize_text).apply(simplify_club_name)
df1_sorted['team_to'] = df1_sorted['team_to'].apply(normalize_text).apply(simplify_club_name)
df1_sorted['transfer_date'] = pd.to_datetime(df1_sorted['transfer_date'], errors='coerce')

df2_sorted['nombre'] = df2_sorted['nombre'].apply(normalize_text).apply(simplify_name)
df2_sorted['fromTeamName'] = df2_sorted['fromTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted['toTeamName'] = df2_sorted['toTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted['startDate'] = pd.to_datetime(df2_sorted['startDate'], errors='coerce')

# Crear un ID temporal basado en mes y año
df1_sorted['ID'] = df1_sorted.apply(lambda row: f"{row['team_from']}_{row['team_to']}_{row['transfer_date'].year}_{row['transfer_date'].month}", axis=1)
df2_sorted['ID'] = df2_sorted.apply(lambda row: f"{row['fromTeamName']}_{row['toTeamName']}_{row['startDate'].year}_{row['startDate'].month}", axis=1)

# Primaria unión por ID
initial_merge = pd.merge(df1_sorted, df2_sorted, on='ID', suffixes=('_df1', '_df2'))

# Unión final basada en similitud
merged_rows = []

for _, merged_row in initial_merge.iterrows():
    name_similarity = fuzz.partial_ratio(merged_row['player_name'], merged_row['nombre'])
    if name_similarity > 80:
        merged_rows.append(merged_row)

# Crear un dataframe final con los resultados
merged_df3 = pd.DataFrame(merged_rows)

# Ver los resultados
print(merged_df3.head())

# Guardar el resultado en un archivo CSV
merged_df3.to_csv('archivo_unido3.csv', index=False)

  row_ind           player_name season transfer_date       team_from  \
0       3            aj cochran  20/21    2021-02-17  phoenix rising   
1       2        aakash sangwan  22/23    2022-06-21          punjab   
2       2         aaren d'silva  20/21    2020-10-28          goa ii   
4       6  aaron amadi-holloway  20/21    2020-10-17   brisbane roar   
7       2  aaron amadi-holloway  22/23    2022-08-05   burton albion   

          team_to market_value transfer_value contract_expiry  \
0     indy eleven       250000              0              NA   
1      chennaiyin       125000              0              NA   
2             goa        75000             NA              NA   
4  sc east bengal        2e+05              0              NA   
7        hereford        2e+05              0              NA   

                                          player_url  query_date  \
0  https://www.transfermarkt.com/aj-cochran/profi...  2024-09-10   
1  https://www.transfermarkt.com/aakash-

In [45]:
merged_df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34063 entries, 0 to 58243
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   row_ind             34063 non-null  object             
 1   player_name         34063 non-null  object             
 2   season              34063 non-null  object             
 3   transfer_date       34063 non-null  datetime64[ns]     
 4   team_from           34063 non-null  object             
 5   team_to             34063 non-null  object             
 6   market_value        34063 non-null  object             
 7   transfer_value      34063 non-null  object             
 8   contract_expiry     34063 non-null  object             
 9   player_url          34063 non-null  object             
 10  query_date          34063 non-null  object             
 11  timestamp           34063 non-null  datetime64[ns, UTC]
 12  blob                31249 non-null  o

In [60]:
merged_df3.head(20)

,row_ind,player_name,season,transfer_date,team_from,team_to,market_value,transfer_value,contract_expiry,player_url,query_date,timestamp,blob,player_id,teams_url,team_id,player_transfer_id,ID,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,endDate,type,value,currency,announceDate,playerId
0,3,aj cochran,20/21,2021-02-17,phoenix rising,indy eleven,250000,0,NA,https://www.transfermarkt.com/aj-cochran/profi...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,306391,https://www.transfermarkt.com/charleston-batte...,4115,202102174115306391,phoenix rising_indy eleven_2021_2,aj cochran,1156285,29992,phoenix rising,27170,indy eleven,0,2021-02-17,2023-02-02,Transfer,0,EUR,,300553
1,2,aakash sangwan,22/23,2022-06-21,punjab,chennaiyin,125000,0,NA,https://www.transfermarkt.com/aakash-sangwan/p...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,503167,https://www.transfermarkt.com/chennaiyin-fc/st...,45299,2022062145299503167,punjab_chennaiyin_2022_6,akash sangwan,1649519,37206,punjab,31882,chennaiyin,0,2022-06-21,2024-06-12,Transfer,0,EUR,,503194
2,2,aaren d'silva,20/21,2020-10-28,goa ii,goa,75000,NA,NA,https://www.transfermarkt.com/aaren-dsilva/pro...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,630382,https://www.transfermarkt.com/hyderabad-fc/sta...,76364,2020102876364630382,goa ii_goa_2020_10,aaren d'silva,1059236,61377,goa ii,31884,goa,0,2020-10-28,2021-06-18,Transfer,0,EUR,,703579
4,6,aaron amadi-holloway,20/21,2020-10-17,brisbane roar,sc east bengal,2e+05,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2020101746842186574,brisbane roar_sc east bengal_2020_10,aaron amadi-holloway,1054694,8314,brisbane roar,15036,sc east bengal,0,2020-10-17,2021-07-23,Free Transfer,0,EUR,,10612
7,2,aaron amadi-holloway,22/23,2022-08-05,burton albion,hereford,2e+05,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2022080546842186574,burton albion_hereford_2022_8,aaron amadi-holloway,1674586,1710,burton albion,36605,hereford,0,2022-08-05,2023-07-12,Transfer,0,EUR,,10612
8,5,aaron amadi-holloway,21/22,2021-07-23,sc east bengal,burton albion,2e+05,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2021072346842186574,sc east bengal_burton albion_2021_7,aaron amadi-holloway,1350346,15036,sc east bengal,1710,burton albion,0,2021-07-23,2022-01-28,Transfer,0,EUR,,10612
9,15,aaron amadi-holloway,11/12,2012-01-01,bath city,bristol city,50000,NA,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2012010146842186574,bath city_bristol city_2012_1,aaron amadi-holloway,1572374,1728,bath city,1655,bristol city,0,2012-01-01,2014-02-12,Back from Loan,0,EUR,,10612
11,16,aaron amadi-holloway,11/12,2011-11-24,bristol city,bath city,50000,NA,2012-06-30,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2011112446842186574,bristol city_bath city_2011_11,aaron amadi-holloway,80644,1655,bristol city,1728,bath city,0,2011-11-24,2012-01-01,Loan,0,,2011-11-26,10612
13,7,aaron arnott,22/23,2022-09-07,raith rovers,brechin city,125000,NA,2024-05-31,https://www.transfermarkt.com/aaron-arnott/pro...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf..

In [61]:


# Funciones auxiliares para la normalización
def normalize_text(text):
    if isinstance(text, str):
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
        return ' '.join(text.lower().strip().split())
    return text

def simplify_name(name):
    parts = name.split()
    return ' '.join([parts[0], parts[-1]]) if len(parts) > 1 else name

def simplify_club_name(club_name):
    ignore_words = {'fc', 'club', 'de', 'the'}
    return ' '.join([word for word in club_name.lower().split() if word not in ignore_words])

# Normalizar y simplificar columnas relevantes
df1_sorted['player_name'] = df1_sorted['player_name'].apply(normalize_text).apply(simplify_name)
df1_sorted['team_from'] = df1_sorted['team_from'].apply(normalize_text).apply(simplify_club_name)
df1_sorted['team_to'] = df1_sorted['team_to'].apply(normalize_text).apply(simplify_club_name)
df1_sorted['transfer_date'] = pd.to_datetime(df1_sorted['transfer_date'], errors='coerce')

df2_sorted['nombre'] = df2_sorted['nombre'].apply(normalize_text).apply(simplify_name)
df2_sorted['fromTeamName'] = df2_sorted['fromTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted['toTeamName'] = df2_sorted['toTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted['startDate'] = pd.to_datetime(df2_sorted['startDate'], errors='coerce')

# Crear un ID temporal basado en los nombres incluidos
df1_sorted['ID'] = df1_sorted.apply(lambda row: f"{row['player_name']}_{row['team_from']}_{row['team_to']}_{row['transfer_date'].year}_{row['transfer_date'].month}", axis=1)
df2_sorted['ID'] = df2_sorted.apply(lambda row: f"{row['nombre']}_{row['fromTeamName']}_{row['toTeamName']}_{row['startDate'].year}_{row['startDate'].month}", axis=1)

# Primaria unión por ID
initial_merge = pd.merge(df1_sorted, df2_sorted, on='ID', suffixes=('_df1', '_df2'))

# Unión final basada en similitud (ya no es muy necesaria si el ID incluye nombres)
merged_rows = []

for _, merged_row in initial_merge.iterrows():
    name_similarity = fuzz.partial_ratio(merged_row['player_name'], merged_row['nombre'])
    if name_similarity > 80:  # Esta parte podría omitirse si el ID ya diferencia lo suficiente
        merged_rows.append(merged_row)

# Crear un dataframe final con los resultados
merged_df4 = pd.DataFrame(merged_rows)

# Ver los resultados
print(merged_df4.head())

# Guardar el resultado en un archivo CSV
merged_df4.to_csv('archivo_unido4.csv', index=False)


  row_ind           player_name season transfer_date       team_from  \
0       3            aj cochran  20/21    2021-02-17  phoenix rising   
1       2         aaren d'silva  20/21    2020-10-28          goa ii   
2       6  aaron amadi-holloway  20/21    2020-10-17   brisbane roar   
3       2  aaron amadi-holloway  22/23    2022-08-05   burton albion   
4       5  aaron amadi-holloway  21/22    2021-07-23  sc east bengal   

          team_to market_value transfer_value contract_expiry  \
0     indy eleven       250000              0              NA   
1             goa        75000             NA              NA   
2  sc east bengal        2e+05              0              NA   
3        hereford        2e+05              0              NA   
4   burton albion        2e+05              0              NA   

                                          player_url  query_date  \
0  https://www.transfermarkt.com/aj-cochran/profi...  2024-09-10   
1  https://www.transfermarkt.com/aaren-d

In [63]:
merged_df4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31075 entries, 0 to 31074
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   row_ind             31075 non-null  object             
 1   player_name         31075 non-null  object             
 2   season              31075 non-null  object             
 3   transfer_date       31075 non-null  datetime64[ns]     
 4   team_from           31075 non-null  object             
 5   team_to             31075 non-null  object             
 6   market_value        31075 non-null  object             
 7   transfer_value      31075 non-null  object             
 8   contract_expiry     31075 non-null  object             
 9   player_url          31075 non-null  object             
 10  query_date          31075 non-null  object             
 11  timestamp           31075 non-null  datetime64[ns, UTC]
 12  blob                29283 non-null  o

In [62]:
merged_df4.head()

,row_ind,player_name,season,transfer_date,team_from,team_to,market_value,transfer_value,contract_expiry,player_url,query_date,timestamp,blob,player_id,teams_url,team_id,player_transfer_id,ID,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,endDate,type,value,currency,announceDate,playerId
0,3,aj cochran,20/21,2021-02-17,phoenix rising,indy eleven,250000,0,NA,https://www.transfermarkt.com/aj-cochran/profi...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,306391,https://www.transfermarkt.com/charleston-batte...,4115,202102174115306391,aj cochran_phoenix rising_indy eleven_2021_2,aj cochran,1156285,29992,phoenix rising,27170,indy eleven,0,2021-02-17,2023-02-02,Transfer,0,EUR,,300553
1,2,aaren d'silva,20/21,2020-10-28,goa ii,goa,75000,NA,NA,https://www.transfermarkt.com/aaren-dsilva/pro...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,630382,https://www.transfermarkt.com/hyderabad-fc/sta...,76364,2020102876364630382,aaren d'silva_goa ii_goa_2020_10,aaren d'silva,1059236,61377,goa ii,31884,goa,0,2020-10-28,2021-06-18,Transfer,0,EUR,,703579
2,6,aaron amadi-holloway,20/21,2020-10-17,brisbane roar,sc east bengal,2e+05,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2020101746842186574,aaron amadi-holloway_brisbane roar_sc east ben...,aaron amadi-holloway,1054694,8314,brisbane roar,15036,sc east bengal,0,2020-10-17,2021-07-23,Free Transfer,0,EUR,,10612
3,2,aaron amadi-holloway,22/23,2022-08-05,burton albion,hereford,2e+05,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2022080546842186574,aaron amadi-holloway_burton albion_hereford_20...,aaron amadi-holloway,1674586,1710,burton albion,36605,hereford,0,2022-08-05,2023-07-12,Transfer,0,EUR,,10612
4,5,aaron amadi-holloway,21/22,2021-07-23,sc east bengal,burton albion,2e+05,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2021072346842186574,aaron amadi-holloway_sc east bengal_burton alb...,aaron amadi-holloway,1350346,15036,sc east bengal,1710,burton albion,0,2021-07-23,2022-01-28,Transfer,0,EUR,,10612


In [ ]:
# Funciones auxiliares para la normalización
def normalize_text(text):
    if isinstance(text, str):
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
        return ' '.join(text.lower().strip().split())
    return text

def simplify_name(name):
    parts = name.split()
    return ' '.join([parts[0], parts[-1]]) if len(parts) > 1 else name

def simplify_club_name(club_name):
    ignore_words = {'fc', 'club', 'de', 'the'}
    return ' '.join([word for word in club_name.lower().split() if word not in ignore_words])

# Normalizar y simplificar columnas relevantes
df1_sorted['player_name'] = df1_sorted['player_name'].apply(normalize_text).apply(simplify_name)
df1_sorted['team_from'] = df1_sorted['team_from'].apply(normalize_text).apply(simplify_club_name)
df1_sorted['team_to'] = df1_sorted['team_to'].apply(normalize_text).apply(simplify_club_name)
df1_sorted['transfer_date'] = pd.to_datetime(df1_sorted['transfer_date'], errors='coerce')

df2_sorted['nombre'] = df2_sorted['nombre'].apply(normalize_text).apply(simplify_name)
df2_sorted['fromTeamName'] = df2_sorted['fromTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted['toTeamName'] = df2_sorted['toTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted['startDate'] = pd.to_datetime(df2_sorted['startDate'], errors='coerce')

# Crear un ID temporal basado en mes y año
df1_sorted['ID'] = df1_sorted.apply(lambda row: f"{row['team_from']}_{row['team_to']}_{row['transfer_date'].year}_{row['transfer_date'].month}", axis=1)
df2_sorted['ID'] = df2_sorted.apply(lambda row: f"{row['fromTeamName']}_{row['toTeamName']}_{row['startDate'].year}_{row['startDate'].month}", axis=1)

# Primaria unión por ID
#initial_merge = pd.merge(df1_sorted, df2_sorted, on='ID', suffixes=('_df1', '_df2'))

# Unión final basada en similitud
merged_rows = []

for _, merged_row in initial_merge.iterrows():
    name_similarity = fuzz.partial_ratio(merged_row['player_name'], merged_row['nombre'])
    if name_similarity > 80:
        merged_rows.append(merged_row)

# Crear un dataframe final con los resultados
merged_df3 = pd.DataFrame(merged_rows)

# Ver los resultados
print(merged_df3.head())

# Guardar el resultado en un archivo CSV
merged_df3.to_csv('archivo_unido3.csv', index=False)

ahora todo con fuzzy

In [67]:
import pandas as pd
import unicodedata
from fuzzywuzzy import fuzz

# Funciones auxiliares para normalización
def normalize_text(text):
    if isinstance(text, str):
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
        return ' '.join(text.lower().strip().split())
    return text

def simplify_name(name):
    parts = name.split()
    return ' '.join([parts[0], parts[-1]]) if len(parts) > 1 else name

def simplify_club_name(club_name):
    ignore_words = {'fc', 'club', 'de', 'the'}
    return ' '.join([word for word in club_name.lower().split() if word not in ignore_words])

# Normalizar y simplificar columnas relevantes
df1_sorted_top1000['player_name'] = df1_sorted_top1000['player_name'].apply(normalize_text).apply(simplify_name)
df1_sorted_top1000['team_from'] = df1_sorted_top1000['team_from'].apply(normalize_text).apply(simplify_club_name)
df1_sorted_top1000['team_to'] = df1_sorted_top1000['team_to'].apply(normalize_text).apply(simplify_club_name)
df1_sorted_top1000['transfer_date'] = pd.to_datetime(df1_sorted_top1000['transfer_date'], errors='coerce')

df2_sorted_top1000['nombre'] = df2_sorted_top1000['nombre'].apply(normalize_text).apply(simplify_name)
df2_sorted_top1000['fromTeamName'] = df2_sorted_top1000['fromTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted_top1000['toTeamName'] = df2_sorted_top1000['toTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted_top1000['startDate'] = pd.to_datetime(df2_sorted_top1000['startDate'], errors='coerce')

# Crear un ID temporal basado en mes y año
#df1_sorted_top1000['ID'] = df1_sorted_top1000.apply(lambda row: f"{row['team_from']}_{row['team_to']}_{row['transfer_date'].year}_{row['transfer_date'].month}", axis=1)
#df2_sorted_top1000['ID'] = df2_sorted_top1000.apply(lambda row: f"{row['fromTeamName']}_{row['toTeamName']}_{row['startDate'].year}_{row['startDate'].month}", axis=1)

# Unión basada en similitud en cada columna relevante
merged_rows = []

for _, row1 in df1_sorted_top1000.iterrows():
    for _, row2 in df2_sorted_top1000.iterrows():
        # Calcular similitud en cada columna
        player_similarity = fuzz.partial_ratio(row1['player_name'], row2['nombre'])
        from_team_similarity = fuzz.partial_ratio(row1['team_from'], row2['fromTeamName'])
        to_team_similarity = fuzz.partial_ratio(row1['team_to'], row2['toTeamName'])
        
        # Comparar fechas solo por año y mes debido a la posible diferencia en días
        date_similarity = (row1['transfer_date'].year == row2['startDate'].year) and (row1['transfer_date'].month == row2['startDate'].month)
        
        # Verificar si la similitud en todas las columnas supera un umbral
        if player_similarity > 60 and from_team_similarity > 80 and to_team_similarity > 80 and date_similarity:
            merged_rows.append(row1.to_dict() | row2.to_dict())

# Crear un dataframe final con los resultados unidos por similitud
merged_df = pd.DataFrame(merged_rows)

# Ver los resultados
print(merged_df.head())

# Guardar el resultado en un archivo CSV
merged_df.to_csv('archivo_unido_similaridad.csv', index=False)


C:\Users\flori\AppData\Local\Temp\ipykernel_14580\1566369469.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_sorted_top1000['player_name'] = df1_sorted_top1000['player_name'].apply(normalize_text).apply(simplify_name)
C:\Users\flori\AppData\Local\Temp\ipykernel_14580\1566369469.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_sorted_top1000['team_from'] = df1_sorted_top1000['team_from'].apply(normalize_text).apply(simplify_club_name)
C:\Users\flori\AppData\Local\Temp\ipykernel_14580\1566369

  row_ind    player_name season transfer_date        team_from      team_to  \
0       3     aj cochran  20/21    2021-02-17   phoenix rising  indy eleven   
1       2     aj cochran  22/23    2023-02-02      indy eleven   charleston   
2       3    aj marcucci  21/22    2021-09-11  ny red bulls ii     new york   
3       2    aj marcucci  24/25    2024-07-21         new york   if gnistan   
4       2  aaren d'silva  20/21    2020-10-28           goa ii          goa   

  market_value transfer_value contract_expiry  \
0       250000              0              NA   
1       175000              0      2023-11-30   
2           NA             NA      2021-11-30   
3       150000             NA      2025-12-31   
4        75000             NA              NA   

                                          player_url  query_date  \
0  https://www.transfermarkt.com/aj-cochran/profi...  2024-09-10   
1  https://www.transfermarkt.com/aj-cochran/profi...  2024-09-10   
2  https://www.transfermar

In [68]:
merged_df.head(20)

,row_ind,player_name,season,transfer_date,team_from,team_to,market_value,transfer_value,contract_expiry,player_url,query_date,timestamp,blob,player_id,teams_url,team_id,player_transfer_id,ID,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,endDate,type,value,currency,announceDate,playerId
0,3,aj cochran,20/21,2021-02-17,phoenix rising,indy eleven,250000,0,NA,https://www.transfermarkt.com/aj-cochran/profi...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,306391,https://www.transfermarkt.com/charleston-batte...,4115,202102174115306391,phoenix rising_indy eleven_2021_2,aj cochran,1156285,29992,phoenix rising,27170,indy eleven,0,2021-02-17,2023-02-02,Transfer,0,EUR,,300553
1,2,aj cochran,22/23,2023-02-02,indy eleven,charleston,175000,0,2023-11-30,https://www.transfermarkt.com/aj-cochran/profi...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,306391,https://www.transfermarkt.com/charleston-batte...,4115,202302024115306391,indy eleven_charleston battery_2023_2,aj cochran,1865206,27170,indy eleven,7866,charleston battery,0,2023-02-02,2023-12-31,Transfer,0,EUR,,300553
2,3,aj marcucci,21/22,2021-09-11,ny red bulls ii,new york,NA,NA,2021-11-30,https://www.transfermarkt.com/aj-marcucci/prof...,2024-09-11,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,884244,https://www.transfermarkt.com/new-york-red-bul...,623,20210911623884244,new york red bulls ii_new york red bulls_2021_9,aj marcucci,2218403,33172,new york red bulls ii,7945,new york red bulls,0,2021-09-11,2024-07-21,Transfer,0,EUR,,743532
3,2,aj marcucci,24/25,2024-07-21,new york,if gnistan,150000,NA,2025-12-31,https://www.transfermarkt.com/aj-marcucci/prof...,2024-09-11,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,884244,https://www.transfermarkt.com/new-york-red-bul...,623,20240721623884244,new york red bulls_if gnistan_2024_7,aj marcucci,2315327,7945,new york red bulls,4812,if gnistan,0,2024-07-21,2024-12-31,Loan,0,EUR,,743532
4,2,aaren d'silva,20/21,2020-10-28,goa ii,goa,75000,NA,NA,https://www.transfermarkt.com/aaren-dsilva/pro...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,630382,https://www.transfermarkt.com/hyderabad-fc/sta...,76364,2020102876364630382,goa ii_goa_2020_10,aaren d'silva,1059236,61377,goa ii,31884,goa,0,2020-10-28,2021-06-18,Transfer,0,EUR,,703579
5,9,aaron amadi-holloway,16/17,2017-01-20,fleetwood,oldham athletic,1e+05,NA,2018-06-30,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2017012046842186574,fleetwood town_oldham athletic afc_2017_1,aaron amadi-holloway,282871,1882,fleetwood town,1665,oldham athletic afc,1,2017-01-20,2018-07-13,Transfer,0,EUR,2017-01-21,10612
6,6,aaron amadi-holloway,20/21,2020-10-17,brisbane roar,sc east bengal,2e+05,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2020101746842186574,brisbane roar_sc east bengal_2020_10,aaron amadi-holloway,1054694,8314,brisbane roar,15036,sc east bengal,0,2020-10-17,2021-07-23,Free Transfer,0,EUR,,10612
7,14,aaron amadi-holloway,13/14,2014-02-12,bristol city,newport county,NA,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2014021246842186574,bristol city_newport county afc_2014_2,aaron amadi-holloway,1437570,1655,bristol city,1768,newport county afc,0,2014-02-12,2014-08-09,Transfer,0,EUR,,10612
8,13,aaron amadi-holloway,14/15,2014-08-09,newport county,wycombe,NA,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-

In [69]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   row_ind             129 non-null    object             
 1   player_name         129 non-null    object             
 2   season              129 non-null    object             
 3   transfer_date       129 non-null    datetime64[ns]     
 4   team_from           129 non-null    object             
 5   team_to             129 non-null    object             
 6   market_value        129 non-null    object             
 7   transfer_value      129 non-null    object             
 8   contract_expiry     129 non-null    object             
 9   player_url          129 non-null    object             
 10  query_date          129 non-null    object             
 11  timestamp           129 non-null    datetime64[ns, UTC]
 12  blob                129 non-null    

In [ ]:
# Realizar una unión completa (full outer join) basada en el ID
merged_full_df = pd.merge(df1_sorted_top1000, df2_sorted_top1000, on='ID', how='outer', suffixes=('_df1', '_df2'))

# Ver los resultados
merged_full_df.head(10)


In [70]:
import pandas as pd
import unicodedata
from fuzzywuzzy import fuzz

# Funciones auxiliares
def normalize_text(text):
    if isinstance(text, str):
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
        return ' '.join(text.lower().strip().split())
    return text

def simplify_name(name):
    parts = name.split()
    return ' '.join([parts[0], parts[-1]]) if len(parts) > 1 else name

def simplify_club_name(club_name):
    ignore_words = {'fc', 'club', 'de', 'the'}
    return ' '.join([word for word in club_name.lower().split() if word not in ignore_words])

# Normalización y simplificación
df1_sorted['player_name'] = df1_sorted['player_name'].apply(normalize_text).apply(simplify_name)
df1_sorted['team_from'] = df1_sorted['team_from'].apply(normalize_text).apply(simplify_club_name)
df1_sorted['team_to'] = df1_sorted['team_to'].apply(normalize_text).apply(simplify_club_name)
df1_sorted['transfer_date'] = pd.to_datetime(df1_sorted['transfer_date'], errors='coerce')

df2_sorted['nombre'] = df2_sorted['nombre'].apply(normalize_text).apply(simplify_name)
df2_sorted['fromTeamName'] = df2_sorted['fromTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted['toTeamName'] = df2_sorted['toTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted['startDate'] = pd.to_datetime(df2_sorted['startDate'], errors='coerce')

# Crear IDs
df1_sorted['ID'] = df1_sorted.apply(lambda row: f"{row['team_from']}_{row['team_to']}_{row['transfer_date'].year}_{row['transfer_date'].month}", axis=1)
df2_sorted['ID'] = df2_sorted.apply(lambda row: f"{row['fromTeamName']}_{row['toTeamName']}_{row['startDate'].year}_{row['startDate'].month}", axis=1)

# Unión basada en similitud
matched_rows = []
unmatched_rows = []

for _, row1 in df1_sorted.iterrows():
    match_found = False  # Flag para verificar si se encontró coincidencia
    for _, row2 in df2_sorted.iterrows():
        player_similarity = fuzz.partial_ratio(row1['player_name'], row2['nombre'])
        from_team_similarity = fuzz.partial_ratio(row1['team_from'], row2['fromTeamName'])
        to_team_similarity = fuzz.partial_ratio(row1['team_to'], row2['toTeamName'])
        date_similarity = (row1['transfer_date'].year == row2['startDate'].year) and (row1['transfer_date'].month == row2['startDate'].month)
        
        if player_similarity > 80 and from_team_similarity > 80 and to_team_similarity > 80 and date_similarity:
            matched_rows.append(row1.to_dict() | row2.to_dict())
            match_found = True
            break  # Salir del segundo bucle cuando se encuentra una coincidencia

    if not match_found:
        unmatched_rows.append(row1.to_dict())

# Crear DataFrames con los resultados
matched_df = pd.DataFrame(matched_rows)
unmatched_df = pd.DataFrame(unmatched_rows)





: 

FUZZY WUZZY CON IDS COMPUESTOS

In [27]:
import pandas as pd
import unicodedata
from fuzzywuzzy import fuzz

# Funciones auxiliares para normalización
def normalize_text(text):
    if isinstance(text, str):
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
        return ' '.join(text.lower().strip().split())
    return text

def simplify_name(name):
    if isinstance(name, str):
        parts = name.split()
        return ' '.join([parts[0], parts[-1]]) if len(parts) > 1 else name
    return name

def simplify_club_name(club_name):
    if isinstance(club_name, str):
        ignore_words = {'fc', 'club', 'de', 'the'}
        return ' '.join([word for word in club_name.lower().split() if word not in ignore_words])
    return club_name

# Normalización y simplificación
df1_sorted_top1000['player_name'] = df1_sorted_top1000['player_name'].apply(normalize_text).apply(simplify_name)
df1_sorted_top1000['team_from'] = df1_sorted_top1000['team_from'].apply(normalize_text).apply(simplify_club_name)
df1_sorted_top1000['team_to'] = df1_sorted_top1000['team_to'].apply(normalize_text).apply(simplify_club_name)
df1_sorted_top1000['transfer_date'] = pd.to_datetime(df1_sorted_top1000['transfer_date'], errors='coerce')

df2_sorted_top1000['nombre'] = df2_sorted_top1000['nombre'].apply(normalize_text).apply(simplify_name)
df2_sorted_top1000['fromTeamName'] = df2_sorted_top1000['fromTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted_top1000['toTeamName'] = df2_sorted_top1000['toTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted_top1000['startDate'] = pd.to_datetime(df2_sorted_top1000['startDate'], errors='coerce')

# Crear IDs compuestos
df1_sorted_top1000['ID_df1'] = df1_sorted_top1000.apply(lambda row: f"{row['player_name']}_{row['team_from']}_{row['team_to']}_{row['transfer_date'].year}_{row['transfer_date'].month}", axis=1)
df2_sorted_top1000['ID_df2'] = df2_sorted_top1000.apply(lambda row: f"{row['nombre']}_{row['fromTeamName']}_{row['toTeamName']}_{row['startDate'].year}_{row['startDate'].month}", axis=1)

# Unión basada en similitud del ID
matched_rows = []
unmatched_rows = []

for _, row1 in df1_sorted_top1000.iterrows():
    match_found = False
    for _, row2 in df2_sorted_top1000.iterrows():
        id_similarity = fuzz.ratio(row1['ID_df1'], row2['ID_df2'])

        if id_similarity > 80:  # Ajusta el umbral según la sensibilidad deseada
            matched_rows.append(row1.to_dict() | row2.to_dict())
            match_found = True
            break
    
    if not match_found:
        # Mantenemos el registro del primer DataFrame en no coicidencias
        unmatched_rows.append(row1.to_dict())

# También revisamos cuál fila de df2 no tiene coincidencia
matched_ids = set(m['ID_df1'] for m in matched_rows)
for _, row2 in df2_sorted_top1000.iterrows():
    if row2['ID_df2'] not in matched_ids:
        unmatched_rows.append(row2.to_dict())

# Crear DataFrames con los resultados
matched_df = pd.DataFrame(matched_rows)
unmatched_df = pd.DataFrame(unmatched_rows)




C:\Users\flori\AppData\Local\Temp\ipykernel_14208\4120098492.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_sorted_top1000['player_name'] = df1_sorted_top1000['player_name'].apply(normalize_text).apply(simplify_name)
C:\Users\flori\AppData\Local\Temp\ipykernel_14208\4120098492.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_sorted_top1000['team_from'] = df1_sorted_top1000['team_from'].apply(normalize_text).apply(simplify_club_name)
C:\Users\flori\AppData\Local\Temp\ipykernel_14208\4120098

In [29]:
matched_df.head(20)

""


In [30]:
matched_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame


,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,endDate,type,value,currency,announceDate,playerId,normalized_nombre,normalized_fromTeamName,normalized_toTeamName,normalized_startDate,ID,ID_df2
1634707,,1442529,<NA>,<NA>,12461,keshla fk,0,2015-01-01,2016-09-30,Free Transfer,0,EUR,,667752,,<NA>,keshla fk,2015-01-01,_<NA>_keshla fk_2015_1,_<NA>_keshla fk_2015_1
1689115,,1453486,12461,keshla fk,<NA>,<NA>,0,2016-09-30,<NA>,Free Agent,0,EUR,,667753,,keshla fk,<NA>,2016-09-30,_keshla fk_<NA>_2016_9,_keshla fk_<NA>_2016_9
1564808,,1442550,<NA>,<NA>,12461,keshla fk,0,2015-01-01,2016-09-30,Free Transfer,0,EUR,,667753,,<NA>,keshla fk,2015-01-01,_<NA>_keshla fk_2015_1,_<NA>_keshla fk_2015_1
1695485,,1453484,12461,keshla fk,<NA>,<NA>,0,2016-09-30,<NA>,Free Agent,0,EUR,,667752,,keshla fk,<NA>,2016-09-30,_keshla fk_<NA>_2016_9,_keshla fk_<NA>_2016_9
521990,bohdan yatselenko,1453100,<NA>,,68937,vik-volun u15,0,2021-07-01,<NA>,Free Transfer,0,EUR,,776941,bohdan yatselenko,,vik-volun u15,2021-07-01,bohdan yatselenko__vik-volun u15_2021_7,bohdan yatselenko__vik-volun u15_2021_7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41527,jong-chan min,1181496,<NA>,,24903,hwaseong,0,2021-02-01,2022-01-01,Free Transfer,0,EUR,,735296,jong-chan min,,hwaseong,2021-02-01,jong-chan min__hwaseong_2021_2,jong-chan min__hwaseong_2021_2
94240,jong-won jang,1542492,22035,ajou university,68105,geoje citizen,0,2021-01-14,2022-01-01,Transfer,0,EUR,,735396,jong-won jang,ajou university,geoje citizen,2021-01-14,jong-won jang_ajou university_geoje citizen_20...,jong-won jang_ajou university_geoje citizen_20...
94248,jong-won jang,1542486,14837,kyung hee university,<NA>,<NA>,0,2015-01-01,2017-01-01,Unknown,0,EUR,,735396,jong-won jang,kyung hee university,<NA>,2015-01-01,jong-won jang_kyung hee university_<NA>_2015_1,jong-won jang_kyung hee university_<NA>_2015_1
41508,jong-won jang,1542490,<NA>,<NA>,22035,ajou university,0,2017-01-01,2021-01-14,Unknown,0,EUR,,735396,jong-won jang,<NA>,ajou university,2017-01-01,jong-won jang_<NA>_ajou university_2017_1,jong-won jang_<NA>_ajou university_2017_1


In [38]:
# Ordenar matched_df por el playerId del df2
matched_df_sorted = matched_df.sort_values(by='playerId')

# Ver los resultados
matched_df_sorted.head(50)

,row_ind,player_name,season,transfer_date,team_from,team_to,market_value,transfer_value,contract_expiry,player_url,query_date,timestamp,blob,player_id,teams_url,team_id,player_transfer_id,ID,ID_df1,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,endDate,type,value,currency,announceDate,playerId,ID_df2
132,2,aaron cresswell,11/12,2011-07-01,tranmere rovers,ipswich,50000,275000,NA,https://www.transfermarkt.com/aaron-cresswell/...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,92571,https://www.transfermarkt.com/west-ham-united/...,379,2011070137992571,aaron cresswell_tranmere rovers_ipswich town_2...,aaron cresswell_tranmere rovers_ipswich_2011_7,aaron cresswell,95602,1661,tranmere rovers,1634,ipswich town,0,2011-07-01,2014-07-04,Transfer,275000,EUR,2011-07-02,8582,aaron cresswell_tranmere rovers_ipswich town_2...
13,15,aaron amadi-holloway,11/12,2012-01-01,bath city,bristol city,50000,NA,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2012010146842186574,aaron amadi-holloway_bristol city_bath city_20...,aaron amadi-holloway_bath city_bristol city_20...,aaron amadi-holloway,80644,1655,bristol city,1728,bath city,0,2011-11-24,2012-01-01,Loan,0,,2011-11-26,10612,aaron amadi-holloway_bristol city_bath city_20...
18,7,aaron amadi-holloway,19/20,2019-07-01,shrewsbury,brisbane roar,1e+05,0,2020-06-30,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2019070146842186574,aaron amadi-holloway_shrewsbury town_brisbane ...,aaron amadi-holloway_shrewsbury_brisbane roar_...,aaron amadi-holloway,768591,1697,shrewsbury town,8314,brisbane roar,0,2019-07-01,2020-10-17,Transfer,0,EUR,,10612,aaron amadi-holloway_shrewsbury town_brisbane ...
17,5,aaron amadi-holloway,21/22,2021-07-23,sc east bengal,burton albion,2e+05,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2021072346842186574,aaron amadi-holloway_sc east bengal_burton alb...,aaron amadi-holloway_sc east bengal_burton alb...,aaron amadi-holloway,1350346,15036,sc east bengal,1710,burton albion,0,2021-07-23,2022-01-28,Transfer,0,EUR,,10612,aaron amadi-holloway_sc east bengal_burton alb...
16,11,aaron amadi-holloway,15/16,2016-05-31,oldham athletic,wycombe,NA,NA,2016-05-31,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2016053146842186574,aaron amadi-holloway_oldham athletic afc_wycom...,aaron amadi-holloway_oldham athletic_wycombe_2...,aaron amadi-holloway,80641,1665,oldham athletic afc,1678,wycombe wanderers,1,2016-05-31,2016-07-01,Back from Loan,0,,2016-02-13,10612,aaron amadi-holloway_oldham athletic afc_wycom...
15,4,aaron amadi-holloway,21/22,2022-01-28,burton albion,barrow,2e+05,NA,2022-06-30,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2022012846842186574,aaron amadi-holloway_burton albion_hereford_20...,aaron amadi-holloway_burton albion_barrow_2022_1,aaron amadi-holloway,1674586,1710,burton albion,36605,hereford,0,2022-08-05,2023-07-12,Transfer,0,EUR,,10612,aaron amadi-holloway_burton albion_hereford_20...
14,14,aaron amadi-holloway,13/14,2014-02-12,bristol city,newport county,NA,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-

In [32]:
import pandas as pd

# Configurar pandas para mostrar todas las columnas
pd.set_option('display.max_columns', None)


In [34]:
unmatched_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 42 columns):
 #   Column                    Non-Null Count  Dtype              
---  ------                    --------------  -----              
 0   row_ind                   1000 non-null   object             
 1   player_name               1000 non-null   object             
 2   season                    1000 non-null   object             
 3   transfer_date             999 non-null    datetime64[ns]     
 4   team_from                 1000 non-null   object             
 5   team_to                   1000 non-null   object             
 6   market_value              1000 non-null   object             
 7   transfer_value            1000 non-null   object             
 8   contract_expiry           1000 non-null   object             
 9   player_url                1000 non-null   object             
 10  query_date                1000 non-null   object             
 11  timestamp        

In [29]:
unmatched_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1809 entries, 0 to 1808
Data columns (total 34 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   row_ind             837 non-null    object             
 1   player_name         837 non-null    object             
 2   season              837 non-null    object             
 3   transfer_date       836 non-null    datetime64[ns]     
 4   team_from           837 non-null    object             
 5   team_to             837 non-null    object             
 6   market_value        837 non-null    object             
 7   transfer_value      837 non-null    object             
 8   contract_expiry     837 non-null    object             
 9   player_url          837 non-null    object             
 10  query_date          837 non-null    object             
 11  timestamp           837 non-null    datetime64[ns, UTC]
 12  blob                802 non-null  

In [35]:
import pandas as pd
import unicodedata
from fuzzywuzzy import fuzz

# Funciones auxiliares para normalización
def normalize_text(text):
    if isinstance(text, str):
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
        return ' '.join(text.lower().strip().split())
    return text

def simplify_name(name):
    if isinstance(name, str):
        parts = name.split()
        return ' '.join([parts[0], parts[-1]]) if len(parts) > 1 else name
    return name

def simplify_club_name(club_name):
    if isinstance(club_name, str):
        ignore_words = {'fc', 'club', 'de', 'the'}
        return ' '.join([word for word in club_name.lower().split() if word not in ignore_words])
    return club_name

# Normalización y simplificación
df1_sorted_top5000['normalized_player_name'] = df1_sorted_top5000['player_name'].apply(normalize_text).apply(simplify_name)
df1_sorted_top5000['normalized_team_from'] = df1_sorted_top5000['team_from'].apply(normalize_text).apply(simplify_club_name)
df1_sorted_top5000['normalized_team_to'] = df1_sorted_top5000['team_to'].apply(normalize_text).apply(simplify_club_name)
df1_sorted_top5000['normalized_transfer_date'] = pd.to_datetime(df1_sorted_top5000['transfer_date'], errors='coerce')

df2_sorted_top5000['normalized_nombre'] = df2_sorted_top5000['nombre'].apply(normalize_text).apply(simplify_name)
df2_sorted_top5000['normalized_fromTeamName'] = df2_sorted_top5000['fromTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted_top5000['normalized_toTeamName'] = df2_sorted_top5000['toTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted_top5000['normalized_startDate'] = pd.to_datetime(df2_sorted_top5000['startDate'], errors='coerce')

# Crear IDs compuestos para coincidencias
df1_sorted_top5000['ID'] = df1_sorted_top5000.apply(lambda row: f"{row['normalized_player_name']}_{row['normalized_team_from']}_{row['normalized_team_to']}_{row['normalized_transfer_date'].year}_{row['normalized_transfer_date'].month}", axis=1)
df2_sorted_top5000['ID'] = df2_sorted_top5000.apply(lambda row: f"{row['normalized_nombre']}_{row['normalized_fromTeamName']}_{row['normalized_toTeamName']}_{row['normalized_startDate'].year}_{row['normalized_startDate'].month}", axis=1)

# Unión basada en similitud del ID
matched_rows = []

for _, row1 in df1_sorted_top5000.iterrows():
    for _, row2 in df2_sorted_top5000.iterrows():
        id_similarity = fuzz.ratio(row1['ID'], row2['ID'])
        
        if id_similarity > 80:  # Ajusta el umbral según la sensibilidad deseada
            # Combinar los diccionarios originales, excluyendo los campos normalizados y el ID
            merged_row = {
                **{k: row1[k] for k in df1_sorted_top5000.columns if not k.startswith('normalized') and k != 'ID'},
                **{k: row2[k] for k in df2_sorted_top5000.columns if not k.startswith('normalized') and k != 'ID'}
            }
            matched_rows.append(merged_row)
            break

# Crear DataFrame con los resultados que solo tienen coincidencias
matched_df = pd.DataFrame(matched_rows)

# Ver los resultados
print("Matches:")
print(matched_df.head())

# Guardar resultados en archivo CSV
matched_df.to_csv('archivo_unido_similaridad.csv', index=False)


C:\Users\flori\AppData\Local\Temp\ipykernel_14208\233059579.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_sorted_top5000['normalized_player_name'] = df1_sorted_top5000['player_name'].apply(normalize_text).apply(simplify_name)
C:\Users\flori\AppData\Local\Temp\ipykernel_14208\233059579.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_sorted_top5000['normalized_team_from'] = df1_sorted_top5000['team_from'].apply(normalize_text).apply(simplify_club_name)
C:\Users\flori\AppData\Local\Temp\ipy

Matches:
  row_ind player_name season transfer_date       team_from         team_to  \
0       6  AJ Cochran  15/16    2016-03-06         Houston    St. Louis FC   
1       3  AJ Cochran  20/21    2021-02-17  Phoenix Rising     Indy Eleven   
2       5  AJ Cochran  17/18    2018-01-24    St. Louis FC       ATL UTD 2   
3       4  AJ Cochran  18/19    2018-12-25       ATL UTD 2  Phoenix Rising   
4       2  AJ Cochran  22/23    2023-02-02     Indy Eleven      Charleston   

  market_value transfer_value contract_expiry  \
0       250000              0              NA   
1       250000              0              NA   
2       150000              0              NA   
3       175000              0              NA   
4       175000              0      2023-11-30   

                                          player_url  ... toTeamId  \
0  https://www.transfermarkt.com/aj-cochran/profi...  ...    32932   
1  https://www.transfermarkt.com/aj-cochran/profi...  ...    27170   
2  https://www.tr

In [37]:


# Configurar pandas para mostrar todas las columnas
pd.set_option('display.max_columns', None)



In [39]:
matched_df.head(20)

,row_ind,player_name,season,transfer_date,team_from,team_to,market_value,transfer_value,contract_expiry,player_url,query_date,timestamp,blob,player_id,teams_url,team_id,player_transfer_id,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,endDate,type,value,currency,announceDate,playerId
0,6,AJ Cochran,15/16,2016-03-06,Houston,St. Louis FC,250000,0,NA,https://www.transfermarkt.com/aj-cochran/profi...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,306391,https://www.transfermarkt.com/charleston-batte...,4115,201603064115306391,AJ Cochran,237878,7944,Houston Dynamo,32932,Saint Louis,1,2016-02-18,<NA>,Transfer,0,,2016-03-27,300553
1,3,AJ Cochran,20/21,2021-02-17,Phoenix Rising,Indy Eleven,250000,0,NA,https://www.transfermarkt.com/aj-cochran/profi...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,306391,https://www.transfermarkt.com/charleston-batte...,4115,202102174115306391,AJ Cochran,1156285,29992,Phoenix Rising FC,27170,Indy Eleven,0,2021-02-17,2023-02-02,Transfer,0,EUR,,300553
2,5,AJ Cochran,17/18,2018-01-24,St. Louis FC,ATL UTD 2,150000,0,NA,https://www.transfermarkt.com/aj-cochran/profi...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,306391,https://www.transfermarkt.com/charleston-batte...,4115,201801244115306391,AJ Cochran,563640,32932,Saint Louis,<NA>,,0,2018-01-24,2018-01-24,Free Agent,0,EUR,,300553
3,4,AJ Cochran,18/19,2018-12-25,ATL UTD 2,Phoenix Rising,175000,0,NA,https://www.transfermarkt.com/aj-cochran/profi...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,306391,https://www.transfermarkt.com/charleston-batte...,4115,201812254115306391,AJ Cochran,695487,62449,Atlanta United II,29992,Phoenix Rising,0,2018-12-25,2021-02-17,Free Transfer,0,EUR,,300553
4,2,AJ Cochran,22/23,2023-02-02,Indy Eleven,Charleston,175000,0,2023-11-30,https://www.transfermarkt.com/aj-cochran/profi...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,306391,https://www.transfermarkt.com/charleston-batte...,4115,202302024115306391,AJ Cochran,1865206,27170,Indy Eleven,7866,Charleston Battery,0,2023-02-02,2023-12-31,Transfer,0,EUR,,300553
5,1,AJ Marcucci,24/25,2024-12-31,IF Gnistan,New York,250000,NA,2024-12-31,https://www.transfermarkt.com/aj-marcucci/prof...,2024-09-11,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,884244,https://www.transfermarkt.com/new-york-red-bul...,623,20241231623884244,AJ Marcucci,2315328,4812,IF Gnistan,7945,New York Red Bulls,0,2024-12-31,<NA>,Back from Loan,0,EUR,,743532
6,4,AJ Marcucci,20/21,2021-04-15,Without Club,NY Red Bulls II,NA,NA,NA,https://www.transfermarkt.com/aj-marcucci/prof...,2024-09-11,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,884244,https://www.transfermarkt.com/new-york-red-bul...,623,20210415623884244,AJ Marcucci,1212041,<NA>,,33172,New York Red Bulls II,0,2021-04-15,2021-09-11,Free Transfer,0,EUR,,743532
7,3,AJ Marcucci,21/22,2021-09-11,NY Red Bulls II,New York,NA,NA,2021-11-30,https://www.transfermarkt.com/aj-marcucci/prof...,2024-09-11,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,884244,https://www.transfermarkt.com/new-york-red-bul...,623,20210911623884244,AJ Marcucci,2218403,33172,New York Red Bulls II,7945,New York Red Bulls,0,2021-09-11,2024-07-21,Transfer,0,EUR,,743532
8,2,AJ Marcucci,24/25,2024-07-21,New York,IF Gnistan,150000,NA,2025-12-31,https://www.transfermarkt.com/aj-marcucci/prof...,2024-09-11,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,884244,https://www.transfermarkt.com/new-york-red-bul...,623,20240721623884244,AJ Marcucci,2315327,7945,New York Red Bulls,4812,IF Gnistan,0,2024-07-21,2024-12-31,Loan,0,EUR,,743532
9,1,Aadil El Gachbour,19/20,2019-07-01,Unknown,L-Montlhéry B,NA,NA,NA,https://www.transfermarkt.com/aadil-el-gachbou...,2024-09

In [40]:
matched_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 694 entries, 0 to 693
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   row_ind             694 non-null    object             
 1   player_name         694 non-null    object             
 2   season              694 non-null    object             
 3   transfer_date       694 non-null    object             
 4   team_from           694 non-null    object             
 5   team_to             694 non-null    object             
 6   market_value        694 non-null    object             
 7   transfer_value      694 non-null    object             
 8   contract_expiry     694 non-null    object             
 9   player_url          694 non-null    object             
 10  query_date          694 non-null    object             
 11  timestamp           694 non-null    datetime64[ns, UTC]
 12  blob                658 non-null    

In [41]:
import pandas as pd
import unicodedata
from fuzzywuzzy import fuzz

# Funciones auxiliares para normalización
def normalize_text(text):
    if isinstance(text, str):
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
        return ' '.join(text.lower().strip().split())
    return text

def simplify_name(name):
    if isinstance(name, str):
        parts = name.split()
        return ' '.join([parts[0], parts[-1]]) if len(parts) > 1 else name
    return name

def simplify_club_name(club_name):
    if isinstance(club_name, str):
        ignore_words = {'fc', 'club', 'de', 'the'}
        return ' '.join([word for word in club_name.lower().split() if word not in ignore_words])
    return club_name

# Normalización y simplificación
df1_sorted_top5000['normalized_player_name'] = df1_sorted_top5000['player_name'].apply(normalize_text).apply(simplify_name)
df1_sorted_top5000['normalized_team_from'] = df1_sorted_top5000['team_from'].apply(normalize_text).apply(simplify_club_name)
df1_sorted_top5000['normalized_team_to'] = df1_sorted_top5000['team_to'].apply(normalize_text).apply(simplify_club_name)
df1_sorted_top5000['normalized_transfer_date'] = pd.to_datetime(df1_sorted_top5000['transfer_date'], errors='coerce')

df2_sorted_top5000['normalized_nombre'] = df2_sorted_top5000['nombre'].apply(normalize_text).apply(simplify_name)
df2_sorted_top5000['normalized_fromTeamName'] = df2_sorted_top5000['fromTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted_top5000['normalized_toTeamName'] = df2_sorted_top5000['toTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted_top5000['normalized_startDate'] = pd.to_datetime(df2_sorted_top5000['startDate'], errors='coerce')

# Crear IDs compuestos para coincidencias
df1_sorted_top5000['ID'] = df1_sorted_top5000.apply(lambda row: f"{row['normalized_player_name']}_{row['normalized_team_from']}_{row['normalized_team_to']}_{row['normalized_transfer_date'].year}_{row['normalized_transfer_date'].month}", axis=1)
df2_sorted_top5000['ID'] = df2_sorted_top5000.apply(lambda row: f"{row['normalized_nombre']}_{row['normalized_fromTeamName']}_{row['normalized_toTeamName']}_{row['normalized_startDate'].year}_{row['normalized_startDate'].month}", axis=1)

# Unión basada en similitud del ID
matched_rows = []

for _, row1 in df1_sorted_top5000.iterrows():
    for _, row2 in df2_sorted_top5000.iterrows():
        id_similarity = fuzz.ratio(row1['ID'], row2['ID'])

        if id_similarity > 80:  # Ajusta el umbral según la sensibilidad deseada
            # Combinar los diccionarios originales, excluyendo los campos normalizados y el ID
            merged_row = {
                **{k: row1[k] for k in df1_sorted_top5000.columns if not k.startswith('normalized') and k != 'ID'},
                **{k: row2[k] for k in df2_sorted_top5000.columns if not k.startswith('normalized') and k != 'ID'},
                'ID_comparación': f"{row1['ID']} / {row2['ID']}"  # Agrego el ID compuesto al resultado
            }
            matched_rows.append(merged_row)
            break

# Crear DataFrame con los resultados que solo tienen coincidencias
matched_df = pd.DataFrame(matched_rows)

# Ver los resultados
print("Matches:")
print(matched_df.head())

# Guardar resultados en archivo CSV
matched_df.to_csv('archivo_unido_similaridad.csv', index=False)


C:\Users\flori\AppData\Local\Temp\ipykernel_14208\2569516023.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_sorted_top5000['normalized_player_name'] = df1_sorted_top5000['player_name'].apply(normalize_text).apply(simplify_name)
C:\Users\flori\AppData\Local\Temp\ipykernel_14208\2569516023.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_sorted_top5000['normalized_team_from'] = df1_sorted_top5000['team_from'].apply(normalize_text).apply(simplify_club_name)
C:\Users\flori\AppData\Local\Temp\i

Matches:
  row_ind player_name season transfer_date       team_from         team_to  \
0       6  AJ Cochran  15/16    2016-03-06         Houston    St. Louis FC   
1       3  AJ Cochran  20/21    2021-02-17  Phoenix Rising     Indy Eleven   
2       5  AJ Cochran  17/18    2018-01-24    St. Louis FC       ATL UTD 2   
3       4  AJ Cochran  18/19    2018-12-25       ATL UTD 2  Phoenix Rising   
4       2  AJ Cochran  22/23    2023-02-02     Indy Eleven      Charleston   

  market_value transfer_value contract_expiry  \
0       250000              0              NA   
1       250000              0              NA   
2       150000              0              NA   
3       175000              0              NA   
4       175000              0      2023-11-30   

                                          player_url  query_date  \
0  https://www.transfermarkt.com/aj-cochran/profi...  2024-09-10   
1  https://www.transfermarkt.com/aj-cochran/profi...  2024-09-10   
2  https://www.transfer

In [42]:
matched_df.head(20)
## las columnas que quiero son player_name, transfer_date, team_from, team_to, market_value, transfer_value, player_id, player_transfer_id, transferId, type, currency, playerId, ID_comparacion

,row_ind,player_name,season,transfer_date,team_from,team_to,market_value,transfer_value,contract_expiry,player_url,query_date,timestamp,blob,player_id,teams_url,team_id,player_transfer_id,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,endDate,type,value,currency,announceDate,playerId,ID_comparación
0,6,AJ Cochran,15/16,2016-03-06,Houston,St. Louis FC,250000,0,NA,https://www.transfermarkt.com/aj-cochran/profi...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,306391,https://www.transfermarkt.com/charleston-batte...,4115,201603064115306391,AJ Cochran,237878,7944,Houston Dynamo,32932,Saint Louis,1,2016-02-18,<NA>,Transfer,0,,2016-03-27,300553,aj cochran_houston_st. louis_2016_3 / aj cochr...
1,3,AJ Cochran,20/21,2021-02-17,Phoenix Rising,Indy Eleven,250000,0,NA,https://www.transfermarkt.com/aj-cochran/profi...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,306391,https://www.transfermarkt.com/charleston-batte...,4115,202102174115306391,AJ Cochran,1156285,29992,Phoenix Rising FC,27170,Indy Eleven,0,2021-02-17,2023-02-02,Transfer,0,EUR,,300553,aj cochran_phoenix rising_indy eleven_2021_2 /...
2,5,AJ Cochran,17/18,2018-01-24,St. Louis FC,ATL UTD 2,150000,0,NA,https://www.transfermarkt.com/aj-cochran/profi...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,306391,https://www.transfermarkt.com/charleston-batte...,4115,201801244115306391,AJ Cochran,563640,32932,Saint Louis,<NA>,,0,2018-01-24,2018-01-24,Free Agent,0,EUR,,300553,aj cochran_st. louis_atl utd 2_2018_1 / aj coc...
3,4,AJ Cochran,18/19,2018-12-25,ATL UTD 2,Phoenix Rising,175000,0,NA,https://www.transfermarkt.com/aj-cochran/profi...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,306391,https://www.transfermarkt.com/charleston-batte...,4115,201812254115306391,AJ Cochran,695487,62449,Atlanta United II,29992,Phoenix Rising,0,2018-12-25,2021-02-17,Free Transfer,0,EUR,,300553,aj cochran_atl utd 2_phoenix rising_2018_12 / ...
4,2,AJ Cochran,22/23,2023-02-02,Indy Eleven,Charleston,175000,0,2023-11-30,https://www.transfermarkt.com/aj-cochran/profi...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,306391,https://www.transfermarkt.com/charleston-batte...,4115,202302024115306391,AJ Cochran,1865206,27170,Indy Eleven,7866,Charleston Battery,0,2023-02-02,2023-12-31,Transfer,0,EUR,,300553,aj cochran_indy eleven_charleston_2023_2 / aj ...
5,1,AJ Marcucci,24/25,2024-12-31,IF Gnistan,New York,250000,NA,2024-12-31,https://www.transfermarkt.com/aj-marcucci/prof...,2024-09-11,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,884244,https://www.transfermarkt.com/new-york-red-bul...,623,20241231623884244,AJ Marcucci,2315328,4812,IF Gnistan,7945,New York Red Bulls,0,2024-12-31,<NA>,Back from Loan,0,EUR,,743532,aj marcucci_if gnistan_new york_2024_12 / aj m...
6,4,AJ Marcucci,20/21,2021-04-15,Without Club,NY Red Bulls II,NA,NA,NA,https://www.transfermarkt.com/aj-marcucci/prof...,2024-09-11,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,884244,https://www.transfermarkt.com/new-york-red-bul...,623,20210415623884244,AJ Marcucci,1212041,<NA>,,33172,New York Red Bulls II,0,2021-04-15,2021-09-11,Free Transfer,0,EUR,,743532,aj marcucci_without_ny red bulls ii_2021_4 / a...
7,3,AJ Marcucci,21/22,2021-09-11,NY Red Bulls II,New York,NA,NA,2021-11-30,https://www.transfermarkt.com/aj-marcucci/prof...,2024-09-11,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,884244,https://www.transfermarkt.com/new-york-red-bul...,623,20210911623884244,AJ Marcucci,2218403,33172,New York Red Bulls II,7945,New York Red Bulls,0,2021-09-11,2024-07-21,Transfer,0,EUR,,743532,aj marcucci_ny red bulls ii_new york_2021_9 / ...
8,2,AJ Marcucci,24/25,2024-07-21,New York,IF Gnistan,150000,NA,2025-12-31,https://www.transfermarkt.com/a

In [43]:
matched_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 694 entries, 0 to 693
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   row_ind             694 non-null    object             
 1   player_name         694 non-null    object             
 2   season              694 non-null    object             
 3   transfer_date       694 non-null    object             
 4   team_from           694 non-null    object             
 5   team_to             694 non-null    object             
 6   market_value        694 non-null    object             
 7   transfer_value      694 non-null    object             
 8   contract_expiry     694 non-null    object             
 9   player_url          694 non-null    object             
 10  query_date          694 non-null    object             
 11  timestamp           694 non-null    datetime64[ns, UTC]
 12  blob                658 non-null    

In [44]:
import pandas as pd
import unicodedata
from fuzzywuzzy import fuzz

# Funciones auxiliares para normalización
def normalize_text(text):
    if isinstance(text, str):
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
        return ' '.join(text.lower().strip().split())
    return ""

def simplify_name(name):
    if isinstance(name, str):
        parts = name.split()
        return ' '.join([parts[0], parts[-1]]) if len(parts) > 1 else name
    return ""

def simplify_club_name(club_name):
    if isinstance(club_name, str):
        ignore_words = {'fc', 'club', 'de', 'the'}
        return ' '.join([word for word in club_name.lower().split() if word not in ignore_words])
    return ""

# Normalización y simplificación
df1_sorted_top5000['normalized_player_name'] = df1_sorted_top5000['player_name'].apply(normalize_text).apply(simplify_name)
df1_sorted_top5000['normalized_team_from'] = df1_sorted_top5000['team_from'].apply(normalize_text).apply(simplify_club_name)
df1_sorted_top5000['normalized_team_to'] = df1_sorted_top5000['team_to'].apply(normalize_text).apply(simplify_club_name)
df1_sorted_top5000['normalized_transfer_date'] = pd.to_datetime(df1_sorted_top5000['transfer_date'], errors='coerce')

df2_sorted_top5000['normalized_nombre'] = df2_sorted_top5000['nombre'].apply(normalize_text).apply(simplify_name)
df2_sorted_top5000['normalized_fromTeamName'] = df2_sorted_top5000['fromTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted_top5000['normalized_toTeamName'] = df2_sorted_top5000['toTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted_top5000['normalized_startDate'] = pd.to_datetime(df2_sorted_top5000['startDate'], errors='coerce')

# Crear IDs compuestos para coincidencias
df1_sorted_top5000['ID'] = df1_sorted_top5000.apply(lambda row: f"{row['normalized_player_name']}_{row['normalized_team_from']}_{row['normalized_team_to']}_{row['normalized_transfer_date'].year}_{row['normalized_transfer_date'].month}", axis=1)
df2_sorted_top5000['ID'] = df2_sorted_top5000.apply(lambda row: f"{row['normalized_nombre']}_{row['normalized_fromTeamName']}_{row['normalized_toTeamName']}_{row['normalized_startDate'].year}_{row['normalized_startDate'].month}", axis=1)

# Definir columnas de interés
cols_df1 = ['player_name', 'transfer_date', 'team_from', 'team_to', 'market_value', 'transfer_value', 'player_id', 'player_transfer_id']
cols_df2 = ['transferId', 'type', 'currency', 'playerId']
cols_common = ['ID_comparacion']

# Unión basada en similitud del ID
matched_rows = []

for _, row1 in df1_sorted_top5000.iterrows():
    for _, row2 in df2_sorted_top5000.iterrows():
        id_similarity = fuzz.ratio(row1['ID'], row2['ID'])

        if id_similarity > 80:  # Ajusta el umbral según la sensibilidad deseada
            # Combinar los diccionarios originales, pero solo con las columnas especificadas
            merged_row = {
                **{k: row1[k] for k in cols_df1 if k in row1},
                **{k: row2[k] for k in cols_df2 if k in row2},
                'ID_comparacion': row1['ID']  # Incluye un solo ID compuesto
            }
            matched_rows.append(merged_row)
            break

# Crear DataFrame con los resultados que solo tienen coincidencias
matched_df2 = pd.DataFrame(matched_rows)

# Ver los resultados
print("Matches:")
print(matched_df2.head())

# Guardar resultados en archivo CSV
matched_df2.to_csv('archivo_unido_similaridad2.csv', index=False)


C:\Users\flori\AppData\Local\Temp\ipykernel_14208\2784782888.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_sorted_top5000['normalized_player_name'] = df1_sorted_top5000['player_name'].apply(normalize_text).apply(simplify_name)
C:\Users\flori\AppData\Local\Temp\ipykernel_14208\2784782888.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_sorted_top5000['normalized_team_from'] = df1_sorted_top5000['team_from'].apply(normalize_text).apply(simplify_club_name)
C:\Users\flori\AppData\Local\Temp\i

Matches:
  player_name transfer_date       team_from         team_to market_value  \
0  AJ Cochran    2016-03-06         Houston    St. Louis FC       250000   
1  AJ Cochran    2021-02-17  Phoenix Rising     Indy Eleven       250000   
2  AJ Cochran    2018-01-24    St. Louis FC       ATL UTD 2       150000   
3  AJ Cochran    2018-12-25       ATL UTD 2  Phoenix Rising       175000   
4  AJ Cochran    2023-02-02     Indy Eleven      Charleston       175000   

  transfer_value player_id  player_transfer_id  transferId           type  \
0              0    306391  201603064115306391      237878       Transfer   
1              0    306391  202102174115306391     1156285       Transfer   
2              0    306391  201801244115306391      563640     Free Agent   
3              0    306391  201812254115306391      695487  Free Transfer   
4              0    306391  202302024115306391     1865206       Transfer   

  currency  playerId                                ID_comparacion  
0 

In [45]:
matched_df2.head()

,player_name,transfer_date,team_from,team_to,market_value,transfer_value,player_id,player_transfer_id,transferId,type,currency,playerId,ID_comparacion
0,AJ Cochran,2016-03-06,Houston,St. Louis FC,250000,0,306391,201603064115306391,237878,Transfer,,300553,aj cochran_houston_st. louis_2016_3
1,AJ Cochran,2021-02-17,Phoenix Rising,Indy Eleven,250000,0,306391,202102174115306391,1156285,Transfer,EUR,300553,aj cochran_phoenix rising_indy eleven_2021_2
2,AJ Cochran,2018-01-24,St. Louis FC,ATL UTD 2,150000,0,306391,201801244115306391,563640,Free Agent,EUR,300553,aj cochran_st. louis_atl utd 2_2018_1
3,AJ Cochran,2018-12-25,ATL UTD 2,Phoenix Rising,175000,0,306391,201812254115306391,695487,Free Transfer,EUR,300553,aj cochran_atl utd 2_phoenix rising_2018_12
4,AJ Cochran,2023-02-02,Indy Eleven,Charleston,175000,0,306391,202302024115306391,1865206,Transfer,EUR,300553,aj cochran_indy eleven_charleston_2023_2


In [ ]:

# Funciones auxiliares para normalización
def normalize_text(text):
    if isinstance(text, str):
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
        return ' '.join(text.lower().strip().split())
    return text

def simplify_name(name):
    if isinstance(name, str):
        parts = name.split()
        return ' '.join([parts[0], parts[-1]]) if len(parts) > 1 else name
    return name

def simplify_club_name(club_name):
    if isinstance(club_name, str):
        ignore_words = {'fc', 'club', 'de', 'the'}
        return ' '.join([word for word in club_name.lower().split() if word not in ignore_words])
    return club_name

# Normalización y simplificación
df1_sorted['normalized_player_name'] = df1_sorted['player_name'].apply(normalize_text).apply(simplify_name)
df1_sorted['normalized_team_from'] = df1_sorted['team_from'].apply(normalize_text).apply(simplify_club_name)
df1_sorted['normalized_team_to'] = df1_sorted['team_to'].apply(normalize_text).apply(simplify_club_name)
df1_sorted['normalized_transfer_date'] = pd.to_datetime(df1_sorted['transfer_date'], errors='coerce')

df2_sorted['normalized_nombre'] = df2_sorted['nombre'].apply(normalize_text).apply(simplify_name)
df2_sorted['normalized_fromTeamName'] = df2_sorted['fromTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted['normalized_toTeamName'] = df2_sorted['toTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted['normalized_startDate'] = pd.to_datetime(df2_sorted['startDate'], errors='coerce')

# Crear IDs compuestos para coincidencias
df1_sorted['ID'] = df1_sorted.apply(lambda row: f"{row['normalized_player_name']}_{row['normalized_team_from']}_{row['normalized_team_to']}_{row['normalized_transfer_date'].year}_{row['normalized_transfer_date'].month}", axis=1)
df2_sorted['ID'] = df2_sorted.apply(lambda row: f"{row['normalized_nombre']}_{row['normalized_fromTeamName']}_{row['normalized_toTeamName']}_{row['normalized_startDate'].year}_{row['normalized_startDate'].month}", axis=1)

# Unión basada en similitud del ID
matched_rows = []

for _, row1 in df1_sorted.iterrows():
    for _, row2 in df2_sorted.iterrows():
        id_similarity = fuzz.ratio(row1['ID'], row2['ID'])
        
        if id_similarity > 80:  # Ajusta el umbral según la sensibilidad deseada
            # Combinar los diccionarios originales, excluyendo los campos normalizados y el ID
            merged_row = {
                **{k: row1[k] for k in df1_sorted.columns if not k.startswith('normalized') and k != 'ID'},
                **{k: row2[k] for k in df2_sorted.columns if not k.startswith('normalized') and k != 'ID'}
            }
            matched_rows.append(merged_row)
            break

# Crear DataFrame con los resultados que solo tienen coincidencias
matched_df = pd.DataFrame(matched_rows)

# Ver los resultados
print("Matches:")
print(matched_df.head())

# Guardar resultados en archivo CSV
matched_df.to_csv('archivo_unido_similaridad.csv', index=False)



In [46]:
import pandas as pd
from fuzzywuzzy import fuzz
import numpy as np
import unicodedata

# Funciones auxiliares
def normalize_text(text):
    if isinstance(text, str):
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
        return ' '.join(text.lower().strip().split())
    return ""

def simplify_name(name):
    if isinstance(name, str):
        parts = name.split()
        return ' '.join([parts[0], parts[-1]]) if len(parts) > 1 else name
    return ""

def simplify_club_name(club_name):
    if isinstance(club_name, str):
        ignore_words = {'fc', 'club', 'de', 'the'}
        return ' '.join([word for word in club_name.lower().split() if word not in ignore_words])
    return ""

def preprocess_df1_chunk(df1):
    df1['normalized_player_name'] = df1['player_name'].apply(normalize_text).apply(simplify_name)
    df1['normalized_team_from'] = df1['team_from'].apply(normalize_text).apply(simplify_club_name)
    df1['normalized_team_to'] = df1['team_to'].apply(normalize_text).apply(simplify_club_name)
    df1['normalized_transfer_date'] = pd.to_datetime(df1['transfer_date'], errors='coerce')
    df1['ID'] = df1.apply(lambda row: f"{row['normalized_player_name']}_{row['normalized_team_from']}_{row['normalized_team_to']}_{row['normalized_transfer_date'].year}_{row['normalized_transfer_date'].month}", axis=1)

def preprocess_df2_chunk(df2):
    df2['normalized_nombre'] = df2['nombre'].apply(normalize_text).apply(simplify_name)
    df2['normalized_fromTeamName'] = df2['fromTeamName'].apply(normalize_text).apply(simplify_club_name)
    df2['normalized_toTeamName'] = df2['toTeamName'].apply(normalize_text).apply(simplify_club_name)
    df2['normalized_startDate'] = pd.to_datetime(df2['startDate'], errors='coerce')
    df2['ID'] = df2.apply(lambda row: f"{row['normalized_nombre']}_{row['normalized_fromTeamName']}_{row['normalized_toTeamName']}_{row['normalized_startDate'].year}_{row['normalized_startDate'].month}", axis=1)

# Procesar DataFrames por chunks
def process_in_chunks(df1, df2, chunk_size=10000):
    matched_rows = []

    df1_chunks = np.array_split(df1, len(df1) // chunk_size + 1)
    df2_chunks = np.array_split(df2, len(df2) // chunk_size + 1)

    for df1_chunk in df1_chunks:
        preprocess_df1_chunk(df1_chunk)

        for df2_chunk in df2_chunks:
            preprocess_df2_chunk(df2_chunk)

            for _, row1 in df1_chunk.iterrows():
                for _, row2 in df2_chunk.iterrows():
                    if pd.notna(row1['ID']) and pd.notna(row2['ID']):
                        id_similarity = fuzz.ratio(row1['ID'], row2['ID'])
                        if id_similarity > 80:  # Umbral de similitud ajustable
                            merged_row = {
                                **{k: row1[k] for k in df1_chunk.columns if not k.startswith('normalized') and k != 'ID'},
                                **{k: row2[k] for k in df2_chunk.columns if not k.startswith('normalized') and k != 'ID'},
                                'ID_comparación': row1['ID']
                            }
                            matched_rows.append(merged_row)

    # Crear DataFrame final con las coincidencias
    return pd.DataFrame(matched_rows)

# Ejecuta el proceso en tus DataFrames cargados `df1` y `df2`
matched_df10 = process_in_chunks(df1, df2)

# Guardar los resultados en un archivo CSV
matched_df10.to_csv('archivo_unido_similaridad10.csv', index=False)


c:\Users\flori\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


: 

DOCU FUZZY  https://pypi.org/project/fuzzywuzzy/

In [53]:
fuzz.ratio("this is a test", "this is a test!")

97

In [54]:
fuzz.partial_ratio("this is a test", "this is a test!")
 

100

In [55]:
fuzz.token_sort_ratio("fuzzy wuzzy was a bear", "wuzzy fuzzy was a bear")

100

In [57]:
fuzz.token_sort_ratio("fuzzy was a bear", "fuzzy fuzzy was a bear")

84

In [58]:
fuzz.token_set_ratio("fuzzy was a bear", "fuzzy fuzzy was a bear")

100